In [1]:
import os
import shutil
import re

import dbf
import pandas as pd
from bs4 import BeautifulSoup
from dbfread import DBF
import numpy as np

from pandas.io.excel import ExcelWriter

In [2]:
path1 = r"c:\ProgramData\AVEVA Plant SCADA 2023\User"
folders = [e for e in os.listdir(path1) if os.path.isdir(os.path.join(path1, e))]

In [4]:
os.path.isdir(os.listdir(path1)[0])

False

In [8]:
os.path.join(path1, 'ff')

'c:\\ProgramData\\AVEVA Plant SCADA 2020 R2\\User\\\\ff'

array([['VO2GGIO98', 'Ind/t', 'STRING', 'ST',
        'Диференційний захист введено (час)'],
       ['VO2GGIO98', 'Ind/stVal', 'STRING', 'ST',
        'Диференційний захист введено (режим)'],
       ['VO3GGIO99', 'Ind/stVal', 'STRING', 'ST',
        'Диференційний захист виведено (режим)'],
       ['VO3GGIO99', 'Ind/t', 'STRING', 'ST',
        'Диференційний захист виведено (час)'],
       ['LO20GGIO96', 'Ind/t', 'STRING', 'ST',
        '1 ступінь газового захисту (час)'],
       ['LO20GGIO96', 'Ind/stVal', 'STRING', 'ST',
        '1 ступінь газового захисту (режим)'],
       ['LO19GGIO95', 'Ind/t', 'STRING', 'ST',
        '2 ступінь газового захисту (відключення) (час)'],
       ['LO19GGIO95', 'Ind/stVal', 'STRING', 'ST',
        '2 ступінь газового захисту (відключення) (режим)'],
       ['LO18GGIO94', 'Ind/t', 'STRING', 'ST',
        '2 ступінь газового захисту (сигнал) (час)'],
       ['LO18GGIO94', 'Ind/stVal', 'STRING', 'ST',
        '2 ступінь газового захисту (сигнал) (режим)']

In [3]:
class ProjectBases:

    def __init__(self, path_project):
        self.path_project = path_project
        self.base_equip = BaseProject(os.path.join(self.path_project, 'equip.DBF'))
        self.base_eqtype = BaseProject(os.path.join(self.path_project, 'eqtype.DBF'))
        self.base_io = BaseProject(os.path.join(self.path_project, 'units.DBF'))
        self.base_tag = BaseProject(os.path.join(self.path_project, 'variable.DBF'))
        self.base_advalm = BaseProject(os.path.join(self.path_project, 'advalm.DBF'))
        self.base_pgdynobj = BaseProject(os.path.join(self.path_project, 'pgdynobj.DBF'))

    def get_dbf(self, name_file):
        db = None
        with dbf.Table(os.path.join(self.path_project, name_file + '.DBF'), codepage='cp1251') as tb:
            db = tb.open(mode=dbf.READ_ONLY)
        return db

    def dataframe(self, name_file):
        return pd.DataFrame(DBF(os.path.join(self.path_project, name_file + '.DBF'), encoding='cp1251'))


class BaseProject:

    def __init__(self, file_name):
        self.filename = file_name

    def get_dbf(self):
        db = None
        with dbf.Table(self.filename, codepage='cp1251') as tb:
            db = tb.open(mode=dbf.READ_ONLY)
        return db

    def to_dbf(self, params_list):
        with dbf.Table(self.filename, codepage='cp1251') as tb:
            tb.open(mode=dbf.READ_WRITE)
            for params in params_list:
                tb.append(params)

    def to_dataframe_dbf(self, df):
        params_list = df.to_dict('records')
        with dbf.Table(self.filename, codepage='cp1251') as tb:
            tb.open(mode=dbf.READ_WRITE)
            for params in params_list:
                tb.append(params)

    def clear_dbf(self):
        with dbf.Table(self.filename, codepage='cp1251') as tb:
            tb.open(mode=dbf.READ_WRITE)
            tb.zap()

    def dataframe(self):
        return pd.DataFrame(DBF(self.filename, encoding='cp1251'))


class Project:
    def __init__(self, name_project, name_cluster='Cluster1', name_server='IOServer1'):
        self.path_projects = r"c:\ProgramData\AVEVA Plant SCADA 2023\User"
        self.name_projects = name_project
        self.path_project = os.path.join(self.path_projects, self.name_projects)

        self.config_setProject_xls = self.get_path_file('setProject.xlsx')
        self.config_libDevice_xls = self.get_path_file('libDevice.xlsx')

        self.type_device_current = ''
        self.cluster = name_cluster
        self.server = name_server
        self.base = ProjectBases(self.path_project)

        self.set_df_xls('df_io', self.config_setProject_xls, 'IO')
        self.set_df_xls('df_equips', self.config_setProject_xls, 'LD')

    def set_df_xls(self, name_df, config_xls, sheet_name):
        self.__dict__[name_df] = None
        try:
            df = pd.read_excel(config_xls, sheet_name=sheet_name)
            df = df.fillna('')
            df = df.astype(str)
            self.__dict__[name_df] = df
            return self.__dict__[name_df]
        except:
            pass

    def create_dbf_from_xls(self, ls_name_file):
        for name_file, key_col in ls_name_file:
            df = self.set_df_xls(f'df_{name_file}', self.config_setProject_xls, name_file)
            df_exist = self.base.dataframe(name_file)

            names = []
            if len(df_exist) != 0:
                names = list(df_exist[key_col].values)

            if df is not None:
                params_list = []
                columns = self.base.get_dbf(name_file).field_names

                for i in range(len(df)):
                    row = df.iloc[i]
                    values = {}
                    for column in columns:
                        val = ''
                        if column in df.columns:
                            val = str(row[column])
                            if val == 'nan':
                                val = ''
                        values[column] = val

                    if values[key_col] not in names:
                        params_list.append(values)
                        names.append(values[key_col])

                with dbf.Table(self.get_path_file(f'{name_file}.DBF'), codepage='cp1251') as tb:
                    tb.open(mode=dbf.READ_WRITE)
                    for params in params_list:
                        tb.append(params)

    def copy_libfile(self, ls_file):
        ls_file = ['libDevice.xlsx', 'setProject.xlsx']

        for name_file in ls_file:
            try:
                shutil.copy(
                    os.path.join('C:', 'libCitectScada', name_file),
                    self.path_project
                )
            except:
                pass

    def create_eqtype(self):
        self.create_dbf_from_xls([('eqtype', 'NAME')])

        template_dev = self.base.dataframe('eqtype')['TEMPLATE'].values
        template_dev
        for dev in template_dev:
            try:
                shutil.copy(
                    os.path.join('C:', 'libCitectScada', dev),
                    self.path_project
                )
            except:
                pass

    def create_io(self):

        if self.df_io is not None:

            # self.df_io['port'] = 'PORT1_BOARD1'
            addr = ''

            params_list = []

            df_tmp = self.base.base_io.dataframe()
            try:
                numb = int(df_tmp.iloc[-1]['NUMBER'])
            except:
                numb = 0

            df_io_exist = self.base.base_io.dataframe()

            names = []
            if len(df_io_exist) != 0:
                names = list(df_io_exist['NAME'].values)
            else:
                numb = 0

            for i in range(len(self.df_io)):
                io = self.df_io.iloc[i]

                if io['name'] not in names:
                    numb += 1

                if io['protocol'] == 'IEC61850N':
                    name_file_iec61850 = f"conf61850_{io['name']}.xml"
                    addr = f"[RUN]:{name_file_iec61850}"
                    self.create_config61850(name_file_iec61850, isLDs=True)

                values = {'SERVER': self.server,
                          'NAME': io['name'],
                          'NUMBER': f'{numb}',
                          'ADDR': addr,
                          'PROTOCOL': io['protocol'],
                          'PORT': io['port'],
                          'MODE': 'Primary',
                          'LOGWRITE': '',
                          'LOGREAD': '',
                          'CACHE': '',
                          'CACHETIME': '',
                          'COMMENT': '',
                          'PROTOCOLID': '',
                          'LINKED': '',
                          'EXTERNDB': '',
                          'DRIVERID': '',
                          'CONNSTRING': '',
                          'REFRESH': '',
                          'TAGPREFIX': '',
                          'LASTUPDATE': '',
                          'REMOTE': '',
                          'REMOTEWRIT': '',
                          'TIME': '',
                          'PERIOD': '',
                          'PHONE': '',
                          'CALLERID': '',
                          'ONCONNECT': '',
                          'ONHANGUP': '',
                          'ONBROWSE': '',
                          'LIVEUPDATE': '',
                          'LASTVARMOD': '',
                          'MEMORY': str(io['memory']),
                          'PERSIST': '',
                          'TAGGEN': '',
                          'TAGGENTEMP': '',
                          'PRIORITY': '1',
                          'BGPOLL': '',
                          'BGPOLLRATE': '',
                          'PERSFREQ': '',
                          'PERSPATH': '',
                          'MINUPDATE': '',
                          'STALEPERIO': '',
                          'READONLY': '',
                          'EXCLUSIVE': ''}

                if values['NAME'] not in names:
                    params_list.append(values)
                    names.append(values['NAME'])

            self.base.base_io.to_dbf(params_list)

    def create_equip(self, isCreateTag=False, typeCreateTag='simple'):
        if self.df_equips is not None:

            # self.df_equips['io'] = self.df_io.iloc[0]['name']  #TODO

            params_list = []

            df_equip_exist = self.base.base_equip.dataframe()

            names = []
            if len(df_equip_exist) != 0:
                names = list(df_equip_exist['NAME'].values)

            for i in range(len(self.df_equips)):
                equip = self.df_equips.iloc[i]

                name_LD = f"{equip['name']}"
                if equip['number'].strip() != '': name_LD += f"_{equip['number']}"
                name_LD1 = name_LD;
                if equip['spec'].strip() != '': name_LD += f"_{equip['spec']}"

                name_equip = name_LD
                if equip['device'].strip() != '': name_equip += f"_{equip['device']}"

                values = {'NAME': name_equip,
                          'CLUSTER': self.cluster,
                          'TYPE': equip['device'],
                          'AREA': '',
                          'LOCATION': '',
                          'COMMENT': equip['comment'],
                          'CUSTOM1': f"{name_LD}\\",
                          # 'CUSTOM1': f"{equip['io']}\\",
                          'CUSTOM2': '',
                          'CUSTOM3': '',
                          'CUSTOM4': '',
                          'CUSTOM5': '',
                          'CUSTOM6': '',
                          'CUSTOM7': '',
                          'CUSTOM8': '',
                          'IODEVICE': equip['io'],
                          'PAGE': '',
                          'HELP': '',
                          'DEFSTATE': '',
                          'SCHEDULED': '',
                          'TAGPREFIX': f"{name_LD}/",
                          'TAGGENLINK': '',
                          'LINKED': '',
                          'EDITCODE': '',
                          'PARAM': '',
                          'COMPOSITE': '',
                          'REFERENCE': '',
                          'DEVSCHED': '',
                          'SCHEDID': '',
                          'ALIAS': '',
                          'CONTENT': '',
                          'HIDDEN': ''}

                if values['NAME'] not in names:
                    params_list.append(values)
                    names.append(values['NAME'])

                if isCreateTag:
                    sheet_name = equip['device'] + '_61850'
                    df_dev = pd.read_excel(self.config_libDevice_xls, sheet_name=sheet_name)
                    df_dev['io'] = equip['io']
                    df_dev['LD'] = name_LD
                    df_dev['LD1'] = name_LD1
                    df_dev['equip'] = values['NAME']
                    self.create_tag(df_dev, type=typeCreateTag)
                    # self.create_tag(df_dev, type='IEC61850')

            self.base.base_equip.to_dbf(params_list)

    def create_tag(self, df, type='simple', replace=None):
        '''

        :param df:
        :param type: 'simple', 'IEC61850'
        :param replace:
        :return:
        '''
        df_tags_exist = self.base.base_tag.dataframe()

        names = []
        if len(df_tags_exist) != 0:
            names = list(df_tags_exist['NAME'].values)

        df_tags = pd.DataFrame()
        if replace and len(df_tags_exist) != 0:
            df_tags = df_tags_exist

        if replace:
            self.base.base_tag.clear_dbf()

        for i in range(len(df)):
            dev = df.iloc[i]
            values = self.create_tag_values(dev, type=type)

            if values['NAME'] not in names:
                df_tags = pd.concat([df_tags, pd.DataFrame([values])], ignore_index=True)
                names.append(values['NAME'])
            else:
                if replace:
                    df_tags.loc[df_tags['NAME'] == values['NAME'], :] = list(values.values())

        self.base.base_tag.to_dataframe_dbf(df_tags)

    def create_tag_values(self, df, type='simple'):
        '''

        :param df:
        :param type: 'simple', 'IEC61850'
        :return:
        '''
        dev = df.fillna('')

        values = {}

        if type == 'simple':
            values = {'NAME': dev['name'],
                      'TYPE': dev['type'],
                      'UNIT': dev['io'],
                      'ADDR': "",
                      'RAW_ZERO': '',
                      'RAW_FULL': '',
                      'ENG_ZERO': '',
                      'ENG_FULL': '',
                      'ENG_UNITS': '',
                      'FORMAT': '',
                      'COMMENT': dev['comment'],
                      'EDITCODE': '',
                      'LINKED': '',
                      'OID': '',
                      'REF1': '',
                      'REF2': '',
                      'DEADBAND': '',
                      'CUSTOM': '',
                      'TAGGENLINK': '',
                      'CLUSTER': self.cluster,
                      'HISTORIAN': 'TRUE',
                      'EQUIP': '',
                      'CUSTOM1': '',
                      'CUSTOM2': '',
                      'CUSTOM3': '',
                      'CUSTOM4': '',
                      'CUSTOM5': '',
                      'CUSTOM6': '',
                      'CUSTOM7': '',
                      'CUSTOM8': '',
                      'ITEM': ''}

        if type == 'IEC61850_104':
            attribs = dev['attribs'].split('/')
            name_tag_attribs = '\\'.join(attribs)
            addr = f"{dev['LD']}/{dev['node']}${dev['fc']}${'$'.join(attribs)}"
            unit = dev['io']
            if dev['attribs'] == '':
                addr = ''
                unit = 'Internal'
            values = {'NAME': f"{dev['LD1']}_{dev['name104']}",
                      'TYPE': dev['type'],
                      'UNIT': unit,
                      'ADDR': '',
                      'RAW_ZERO': f"{dev['RAW_ZERO']}",
                      'RAW_FULL': f"{dev['RAW_FULL']}",
                      'ENG_ZERO': f"{dev['ENG_ZERO']}",
                      'ENG_FULL': f"{dev['ENG_FULL']}",
                      'ENG_UNITS': f"{dev['ENG_UNITS']}",
                      'FORMAT': rf"{dev['FORMAT']}",
                      # 'FORMAT': "###.##",
                      'COMMENT': dev['comment'],
                      'EDITCODE': '',
                      'LINKED': '',
                      'OID': '',
                      'REF1': '',
                      'REF2': '',
                      'DEADBAND': '',
                      'CUSTOM': '',
                      'TAGGENLINK': '',
                      'CLUSTER': self.cluster,
                      'HISTORIAN': 'TRUE',
                      'EQUIP': dev['equip'],
                      'CUSTOM1': '',
                      'CUSTOM2': '',
                      'CUSTOM3': '',
                      'CUSTOM4': '',
                      'CUSTOM5': '',
                      'CUSTOM6': '',
                      'CUSTOM7': '',
                      'CUSTOM8': '',
                      'ITEM': f"{dev['node']}\\{name_tag_attribs}"}

        if type == 'IEC61850':
            attribs = dev['attribs'].split('/')
            name_tag_attribs = '\\'.join(attribs)
            addr = f"{dev['LD']}/{dev['node']}${dev['fc']}${'$'.join(attribs)}"
            unit = dev['io']
            if dev['attribs'] == '':
                addr = ''
                unit = 'Internal'
            values = {'NAME': f"{dev['LD']}\\{dev['node']}\\{name_tag_attribs}",
                      'TYPE': dev['type'],
                      'UNIT': unit,
                      'ADDR': addr,
                      'RAW_ZERO': '',
                      'RAW_FULL': '',
                      'ENG_ZERO': '',
                      'ENG_FULL': '',
                      'ENG_UNITS': '',
                      'FORMAT': '',
                      'COMMENT': dev['comment'],
                      'EDITCODE': '',
                      'LINKED': '',
                      'OID': '',
                      'REF1': '',
                      'REF2': '',
                      'DEADBAND': '',
                      'CUSTOM': '',
                      'TAGGENLINK': '',
                      'CLUSTER': self.cluster,
                      'HISTORIAN': 'TRUE',
                      'EQUIP': dev['equip'],
                      'CUSTOM1': '',
                      'CUSTOM2': '',
                      'CUSTOM3': '',
                      'CUSTOM4': '',
                      'CUSTOM5': '',
                      'CUSTOM6': '',
                      'CUSTOM7': '',
                      'CUSTOM8': '',
                      'ITEM': f"{dev['node']}\\{name_tag_attribs}"}

        return values

    def create_config61850(self, name_file, isLDs=False):

        s1 = ""

        if isLDs:
            lds = []
            if self.df_equips is not None:
                for i in range(len(self.df_equips)):
                    equip = self.df_equips.iloc[i]

                    name_LD = f"{equip['name']}"
                    if equip['number'].strip() != '': name_LD += f"_{equip['number']}"
                    if equip['spec'].strip() != '': name_LD += f"_{equip['spec']}"
                    lds.append(name_LD)
                    s1 += f"""
                            <LogicalDevice Name="{name_LD}">
                                  <BRCB>brcbEV1</BRCB>
                                  <BRCB>brcbEV2</BRCB>
                                  <URCB>urcbEV1</URCB>
                               </LogicalDevice>

                        """
            else:
                f"""
                    <LogicalDevice Name="Relay">
                          <BRCB>brcbEV1</BRCB>
                          <BRCB>brcbEV2</BRCB>
                          <URCB>urcbEV1</URCB>
                       </LogicalDevice>
                """

        xml_doc = f"""
            <ScadaDevice xmlns="http://www.schneider-electric.com/SCADA/Drivers/IEC61850/DeviceConfig/v1/">
               <IPConnection>
                  <IP>127.0.0.1</IP>
                  <Port>102</Port>
               </IPConnection>
               <IED>TEMPLATE</IED>
               {s1}
            </ScadaDevice>
            """

        soup = BeautifulSoup(xml_doc, 'lxml-xml')
        xml = soup.prettify()
        with open(self.get_path_file(name_file), "w", encoding='utf-8') as file:
            file.write(xml)

    def get_path_file(self, name_file):
        return os.path.join(self.path_project, name_file)

# project = Project('test61850_2')
# project.copy_libfile(['libDevice.xlsx', 'setProject.xlsx'])
# project.create_dbf_from_xls([('boards', 'NAME'), ('ports', 'NAME')])
# 
# project.create_eqtype()
# project.create_io()

In [4]:
project = Project('FabrichnayaDSP_main')

In [6]:
df = project.base.base_advalm.dataframe()
df

,TAG,NAME,DESC,EXPR,CATEGORY,HELP,PRIV,AREA,COMMENT,SEQUENCE,...,CUSTOM8,CLUSTER,PAGING,PAGINGGRP,EDITCODE,LINKED,TAGGENLINK,EQUIP,ITEM,HISTORIAN
0,A0001,Стрiлка 1/3,Видана команда на переведення в ПЛЮС,"TagBitF(502, 0)",4,,,,,,...,,,,,,,,,,
1,A0002,Стрiлка 1/3,Видана команда на переведення в МIНУС,"TagBitF(502, 1)",4,,,,,,...,,,,,,,,,,
2,A0003,Стрiлка 1/3,Видана команда на допомiжне переведення,"TagBitF(502, 2)",4,,,,,,...,,,,,,,,,,
3,A0004,Стрiлка 1/3,Стрiлка переведена в ПЛЮС,"TagBitF(502, 3)",50,,,,,,...,,,,,,,,,,
4,A0005,Стрiлка 1/3,Стрiлка переведена в МIНУС,"TagBitF(502, 4)",50,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3285,A3318,Предвх. св. ЧЗ109А,Несправнiсть в ланцюзi жовтого вогня усунена,"RegGetValueMidBit(RegGetValueF(995), 0, 8) <> 4",10,,,,,,...,,,,,,,,,,
3286,A3319,Предвх. св. ЧЗ109А,Несправнiсть в ланцюзi зеленого вогня,"RegGetValueMidBit(RegGetValueF(995), 0, 8) = 5",10,,,,,,...,,,,,,,,,,
3287,A3320,Предвх. св. ЧЗ109А,Несправнiсть в ланцюзi зеленого вогня усунена,"RegGetValueMidBit(RegGetValueF(995), 0, 8) <> 5",10,,,,,,...,,,,,,,,,,
3288,A3321,Предвх. св. ЧЗ109А,Включений жовтий вогонь (при несправностi свiт...,"RegGetValueMidBit(RegGetValueF(995), 0, 8) = 6",10,,,,,,...,,,,,,,,,,


In [9]:
vc = df['NAME'].value_counts()

In [15]:
df1 = df[df['NAME'].str.startswith('Стрiлка')]
df1

,TAG,NAME,DESC,EXPR,CATEGORY,HELP,PRIV,AREA,COMMENT,SEQUENCE,...,CUSTOM8,CLUSTER,PAGING,PAGINGGRP,EDITCODE,LINKED,TAGGENLINK,EQUIP,ITEM,HISTORIAN
0,A0001,Стрiлка 1/3,Видана команда на переведення в ПЛЮС,"TagBitF(502, 0)",4,,,,,,...,,,,,,,,,,
1,A0002,Стрiлка 1/3,Видана команда на переведення в МIНУС,"TagBitF(502, 1)",4,,,,,,...,,,,,,,,,,
2,A0003,Стрiлка 1/3,Видана команда на допомiжне переведення,"TagBitF(502, 2)",4,,,,,,...,,,,,,,,,,
3,A0004,Стрiлка 1/3,Стрiлка переведена в ПЛЮС,"TagBitF(502, 3)",50,,,,,,...,,,,,,,,,,
4,A0005,Стрiлка 1/3,Стрiлка переведена в МIНУС,"TagBitF(502, 4)",50,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,A1074,Стрiлка 302,Знята с макета,"NOT TagBitF(676, 7)",50,,,,,,...,,,,,,,,,,
1074,A1075,Стрiлка 302,Отриман контроль положення ПЛЮС,"TagBitF(676, 9)",50,,,,,,...,,,,,,,,,,
1075,A1076,Стрiлка 302,Втрачен контроль положення ПЛЮС,"NOT TagBitF(676, 9)",50,,,,,,...,,,,,,,,,,
1076,A1077,Стрiлка 302,Отриман контроль положения МIНУС,"TagBitF(676, 10)",50,,,,,,...,,,,,,,,,,


In [16]:
st1 = df1['NAME'].unique()
st1

array(['Стрiлка 1/3', 'Стрiлка 5', 'Стрiлка 9', 'Стрiлка 11/13',
       'Стрiлка 14/16', 'Стрiлка 15', 'Стрiлка 17', 'Стрiлка 18',
       'Стрiлка 19', 'Стрiлка 20', 'Стрiлка 22/24', 'Стрiлка 25',
       'Стрiлка 26/28', 'Стрiлка 27', 'Стрiлка 29/31', 'Стрiлка 34',
       'Стрiлка 32', 'Стрiлка 33/35', 'Стрiлка 36', 'Стрiлка 37',
       'Стрiлка 38', 'Стрiлка 39', 'Стрiлка 41', 'Стрiлка 43',
       'Стрiлка 47', 'Стрiлка 49', 'Стрiлка 51', 'Стрiлка 53',
       'Стрiлка 101', 'Стрiлка 107', 'Стрiлка 117/119', 'Стрiлка 121',
       'Стрiлка 201', 'Стрiлка 202', 'Стрiлка 203', 'Стрiлка 204/206',
       'Стрiлка 208', 'Стрiлка 212', 'Стрiлка 4', 'Стрiлка 6',
       'Стрiлка 8', 'Стрiлка 12', 'Стрiлка 84', 'Стрiлка 98',
       'Стрiлка 100', 'Стрiлка 102', 'Стрiлка 103', 'Стрiлка 104',
       'Стрiлка 105', 'Стрiлка 106', 'Стрiлка 108', 'Стрiлка 109',
       'Стрiлка 110', 'Стрiлка 111', 'Стрiлка 112', 'Стрiлка 113',
       'Стрiлка 114', 'Стрiлка 115', 'Стрiлка 116', 'Стрiлка 120',
       

In [20]:
st12 = df1['NAME'].apply(lambda x: x[8:])
ls_str = st12.unique().tolist()
ls_str

['1/3',
 '5',
 '9',
 '11/13',
 '14/16',
 '15',
 '17',
 '18',
 '19',
 '20',
 '22/24',
 '25',
 '26/28',
 '27',
 '29/31',
 '34',
 '32',
 '33/35',
 '36',
 '37',
 '38',
 '39',
 '41',
 '43',
 '47',
 '49',
 '51',
 '53',
 '101',
 '107',
 '117/119',
 '121',
 '201',
 '202',
 '203',
 '204/206',
 '208',
 '212',
 '4',
 '6',
 '8',
 '12',
 '84',
 '98',
 '100',
 '102',
 '103',
 '104',
 '105',
 '106',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '120',
 '122',
 '124',
 '128',
 '130',
 '132',
 '134',
 '136',
 '138/140',
 '142',
 '144',
 '146',
 '148',
 '160',
 '162',
 '164',
 '166',
 '302']

In [22]:
ls_names = ['Стрiлка', 'Вих. св.', 'Предвх. св.', 'Ман. св.', 'Стр. дiл.', 'Безстр. дiл.', 'Шлях', 'Тупик']
ds = {}

for s in ls_names:
    df1 = df[df['NAME'].str.startswith(s)]
    st1 = df1['NAME'].unique()
    st2 = df1['NAME'].apply(lambda x: x[len(s) + 1:])
    ls_str = st2.unique().tolist()
    ds[s] = ls_str


In [27]:
s1 = 'RegGetValueMidBit(RegGetValueF(994), 0, 8) = 1'
s2 = 'AlmSpFree(814, 8) = 0'
s3 = 'TagBitF(628, 3)'


In [92]:
def f1(row):
    s1 = row['EXPR']
    if s1.startswith('RegGetValueMidBit'):
        return s1[31:].split(')')[0]
    if 'TagBit' in s1:
        return s1.split('(')[1][:-1].split(',')[0]
    if s1.startswith('Alm'):
        ss2 = s1.split('(')
        ss3 = ss2[1].split(')')
        return ss3[0].split(',')[0]

In [78]:
df2= df[['NAME', 'DESC', 'EXPR']].copy()
df2

,NAME,DESC,EXPR
0,Стрiлка 1/3,Видана команда на переведення в ПЛЮС,"TagBitF(502, 0)"
1,Стрiлка 1/3,Видана команда на переведення в МIНУС,"TagBitF(502, 1)"
2,Стрiлка 1/3,Видана команда на допомiжне переведення,"TagBitF(502, 2)"
3,Стрiлка 1/3,Стрiлка переведена в ПЛЮС,"TagBitF(502, 3)"
4,Стрiлка 1/3,Стрiлка переведена в МIНУС,"TagBitF(502, 4)"
...,...,...,...
3285,Предвх. св. ЧЗ109А,Несправнiсть в ланцюзi жовтого вогня усунена,"RegGetValueMidBit(RegGetValueF(995), 0, 8) <> 4"
3286,Предвх. св. ЧЗ109А,Несправнiсть в ланцюзi зеленого вогня,"RegGetValueMidBit(RegGetValueF(995), 0, 8) = 5"
3287,Предвх. св. ЧЗ109А,Несправнiсть в ланцюзi зеленого вогня усунена,"RegGetValueMidBit(RegGetValueF(995), 0, 8) <> 5"
3288,Предвх. св. ЧЗ109А,Включений жовтий вогонь (при несправностi свiт...,"RegGetValueMidBit(RegGetValueF(995), 0, 8) = 6"


In [79]:
df2['reg'] = None

In [93]:
df2['reg'] = df2.apply(f1, axis=1)

In [94]:
df2

,NAME,DESC,EXPR,reg
0,Стрiлка 1/3,Видана команда на переведення в ПЛЮС,"TagBitF(502, 0)",502
1,Стрiлка 1/3,Видана команда на переведення в МIНУС,"TagBitF(502, 1)",502
2,Стрiлка 1/3,Видана команда на допомiжне переведення,"TagBitF(502, 2)",502
3,Стрiлка 1/3,Стрiлка переведена в ПЛЮС,"TagBitF(502, 3)",502
4,Стрiлка 1/3,Стрiлка переведена в МIНУС,"TagBitF(502, 4)",502
...,...,...,...,...
3285,Предвх. св. ЧЗ109А,Несправнiсть в ланцюзi жовтого вогня усунена,"RegGetValueMidBit(RegGetValueF(995), 0, 8) <> 4",995
3286,Предвх. св. ЧЗ109А,Несправнiсть в ланцюзi зеленого вогня,"RegGetValueMidBit(RegGetValueF(995), 0, 8) = 5",995
3287,Предвх. св. ЧЗ109А,Несправнiсть в ланцюзi зеленого вогня усунена,"RegGetValueMidBit(RegGetValueF(995), 0, 8) <> 5",995
3288,Предвх. св. ЧЗ109А,Включений жовтий вогонь (при несправностi свiт...,"RegGetValueMidBit(RegGetValueF(995), 0, 8) = 6",995


In [107]:
df3 = df2.iloc[df2['NAME'].drop_duplicates().index]
df4 = df3[['NAME', 'reg']]
df4

,NAME,reg
0,Стрiлка 1/3,502
14,Стрiлка 5,506
28,Стрiлка 9,508
42,Стрiлка 11/13,510
56,Стрiлка 14/16,514
...,...,...
3245,Предвх. св. НЗ108,991
3254,Предвх. св. НЗ108А,992
3263,Предвх. св. НЗ109А,993
3272,Предвх. св. ЧЗ108А,994


In [114]:
# writer = pd.ExcelWriter('reg_dev_fabr.xlsx', engine='xlsxwriter')
df4.to_excel('reg_dev_fabr.xlsx', index=False)

In [13]:
# project = Project('test61850_2')
project = Project('Vostochnaya_v3')
project.copy_libfile(['libDevice.xlsx', 'setProject.xlsx'])
# project.create_dbf_from_xls([('boards', 'NAME'), ('ports', 'NAME')])

project.create_eqtype()
project.create_io()
# project.create_equip()
project.create_equip(isCreateTag=True, typeCreateTag='IEC61850_104')

In [12]:
# project = Project('test61850_2')
project = Project('Vostochnaya_v3')
project.base.base_io.clear_dbf()
project.base.base_equip.clear_dbf()
project.base.base_tag.clear_dbf()

In [16]:
sheet_name = 'P3F30_line6_61850'
df_dev = pd.read_excel(project.config_libDevice_xls, sheet_name=sheet_name)
df_dev

,node,attribs,type,fc,comment,RAW_ZERO,RAW_FULL,ENG_ZERO,ENG_FULL,ENG_UNITS,FORMAT
0,VO3GGIO99,Ind/t,STRING,ST,автомат кіл 3Uo вимкнено (час),NaN,NaN,NaN,NaN,NaN,NaN
1,VO3GGIO99,Ind/q,STRING,ST,автомат кіл 3Uo вимкнено (якість),NaN,NaN,NaN,NaN,NaN,NaN
2,VO3GGIO99,Ind/stVal,STRING,ST,автомат кіл 3Uo вимкнено (режим),NaN,NaN,NaN,NaN,NaN,NaN
3,VO4GGIO100,Ind/stVal,STRING,ST,автомат оперативних кіл комірки вимкнено (режим),NaN,NaN,NaN,NaN,NaN,NaN
4,VO4GGIO100,Ind/t,STRING,ST,автомат оперативних кіл комірки вимкнено (час),NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
93,U3ppMMXU5,PPV/phsCA/t,STRING,MX,напруга фаз CA (час),NaN,NaN,NaN,NaN,NaN,NaN
94,U3ppMMXU5,PPV/phsCA/q,STRING,MX,напруга фаз CA (якість),NaN,NaN,NaN,NaN,NaN,NaN
95,Io1MMXU11,A/neut/cVal/mag/f,REAL,MX,струм 3Io (вимірювання),0.0,20.0,0.0,100.0,Amps,###.##
96,Io1MMXU11,A/neut/t,STRING,MX,струм 3Io (час),NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
project = Project('test61850_1')
# project.copy_libfile(['libDevice.xlsx', 'setProject.xlsx'])
project.create_dbf_from_xls([('boards', 'NAME'), ('ports', 'NAME')])

In [40]:
df1 = pd.DataFrame([{'x1': 1, 'x2': True}])
df1

,x1,x2
0,1,True


In [70]:
# project = Project('Vostochnaya_v3')
project = Project('tmp1')
df1 = project.base.base_pgdynobj.dataframe()
df1

,PAGE,TYPE,AN,GROUP,SZX,SZXMINPC,SZXMAXPC,SZXORG,SZXMODE,SZY,...,FILMODE,DISABLED,DISMODE,GREY_STYLE,DESC,COMMENT,AREA,PRIV,MSG_LOG,DEVICE
0,!ProcessAnalystPopup,0x000D,3,4,,,,,,,...,,,,,,,,,,
1,!Splash,0x0005,14,,,,,,,,...,,,,,,,,,,
2,!Shutdown,0x0005,5,,,,,,,,...,,,,,,,,,,
3,Disabled,0x0007,803,98,,,,,,,...,,,,,,,,,,
4,Hardware,0x0007,803,998,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10736,Summary1,0x0005,704,126,,0,100,,,,...,,,,,,@(Filtered by equipment status),,,,
10737,my,0x0005,616,,,,,,,,...,,,,,,,,,,
10738,my,0x0005,618,,,,,,,,...,,,,,,,,,,
10739,my,0x0005,620,,,,,,,,...,,,,,,,,,,


In [71]:
# df1['TXT_STR'].iloc[-1] = r'Cell_2_TH6\fMMXU8\Hz\mag\f'
# df1[df1['TXT_STR'] == r'Cell_2_TH6\UV2PTUV2\Str\general']
df1[df1['TXT_STR'] == r'Cell_2_TH_6kV\VO3GGIO99\Ind\stVal']


,PAGE,TYPE,AN,GROUP,SZX,SZXMINPC,SZXMAXPC,SZXORG,SZXMODE,SZY,...,FILMODE,DISABLED,DISMODE,GREY_STYLE,DESC,COMMENT,AREA,PRIV,MSG_LOG,DEVICE


In [59]:
df1.loc[10740, 'TXT_STR'] = r'Cell_2_TH_6kV\VO4GGIO100\Ind\stVal'

In [61]:
project.base.base_pgdynobj.clear_dbf()
project.base.base_pgdynobj.to_dataframe_dbf(df1)

In [4]:
sr = df1[df1['NAME'].str.contains(r'stVal|general|ctlVal', regex=True)]

In [5]:
sr

,NAME,TYPE,UNIT,ADDR,RAW_ZERO,RAW_FULL,ENG_ZERO,ENG_FULL,ENG_UNITS,FORMAT,...,EQUIP,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,CUSTOM5,CUSTOM6,CUSTOM7,CUSTOM8,ITEM
2,Cell_2_TH6\VO3GGIO99\Ind\stVal,STRING,Internal,,,,,,,,...,Cell_2_TH6_P3U30_protectTN,,,,,,,,,VO3GGIO99\Ind\stVal
3,Cell_2_TH6\VO4GGIO100\Ind\stVal,STRING,Internal,,,,,,,,...,Cell_2_TH6_P3U30_protectTN,,,,,,,,,VO4GGIO100\Ind\stVal
6,Cell_2_TH6\Obj2CSWI2\Pos\stVal,STRING,Internal,,,,,,,,...,Cell_2_TH6_P3U30_protectTN,,,,,,,,,Obj2CSWI2\Pos\stVal
9,Cell_2_TH6\Obj3CSWI3\Pos\Oper\ctlVal,DIGITAL,Internal,,,,,,,,...,Cell_2_TH6_P3U30_protectTN,,,,,,,,,Obj3CSWI3\Pos\Oper\ctlVal
12,Cell_2_TH6\local_Obj2CSWI2_Pos_stVal\,INT,Internal,,,,,,,,...,Cell_2_TH6_P3U30_protectTN,,,,,,,,,local_Obj2CSWI2_Pos_stVal\
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2370,RZA_4_RPN2\ATCC1\LTCBlkAHi\stVal,STRING,Internal,,,,,,,,...,RZA_4_RPN2_Tapcon_RPN,,,,,,,,,ATCC1\LTCBlkAHi\stVal
2373,RZA_4_RPN2\ATCC1\TapPosBS\stVal,STRING,Internal,,,,,,,,...,RZA_4_RPN2_Tapcon_RPN,,,,,,,,,ATCC1\TapPosBS\stVal
2379,RZA_4_RPN2\ATCC1\TapChg\ctlVal,STRING,Internal,,,,,,,,...,RZA_4_RPN2_Tapcon_RPN,,,,,,,,,ATCC1\TapChg\ctlVal
2382,RZA_4_RPN2\GGIO1\GPO1\stVal,STRING,Internal,,,,,,,,...,RZA_4_RPN2_Tapcon_RPN,,,,,,,,,GGIO1\GPO1\stVal


In [19]:
df1['NAME'].str.extract(r"(\S+$)")

,0
0,Cell_2_TH6\VO3GGIO99\Ind\t
1,Cell_2_TH6\VO3GGIO99\Ind\q
2,Cell_2_TH6\VO3GGIO99\Ind\stVal
3,Cell_2_TH6\VO4GGIO100\Ind\stVal
4,Cell_2_TH6\VO4GGIO100\Ind\t
...,...
2383,RZA_4_RPN2\GGIO1\GPO1\t
2384,RZA_4_RPN2\GGIO1\GPO1\q
2385,RZA_4_RPN2\GGIO1\GPO2\stVal
2386,RZA_4_RPN2\GGIO1\GPO2\t


In [27]:
s1 = sr['NAME'].values[1]
s1

'Cell_2_TH6\\U3ppMMXU5\\PPV\\phsAB\\cVal\\mag\\f'

In [23]:
'\\mag' in r'Cell_2_TH6\\U3ppMMXU5\\PPV\\phsAB\\cVal\\mag\\f'


True

In [14]:
project = Project('Vostochnaya_v5')
df1 = project.base.base_tag.dataframe()
df1

,NAME,TYPE,UNIT,ADDR,RAW_ZERO,RAW_FULL,ENG_ZERO,ENG_FULL,ENG_UNITS,FORMAT,...,EQUIP,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,CUSTOM5,CUSTOM6,CUSTOM7,CUSTOM8,ITEM
0,Cell_2_F,REAL,Internal,,0.0,100.0,30.0,70.0,Hz,###.##,...,Cell_2_TH6_P3U30_protectTN,,,,,,,,,fMMXU8\Hz\mag\f
1,Cell_2_FailureTN,DIGITAL,Internal,,,,,,,,...,Cell_2_TH6_P3U30_protectTN,,,,,,,,,NSVPTOV1\Health\stVal
2,Cell_2_Oper_sw_off,INT,Internal,,,,,,,,...,Cell_2_TH6_P3U30_protectTN,,,,,,,,,Obj2CSWI2\Pos\stVal
3,Cell_2_Command_VE,INT,Internal,,,,,,,,...,Cell_2_TH6_P3U30_protectTN,,,,,,,,,Obj2CSWI2\Pos\Oper\ctlVal
4,Cell_2_Command_ZAZ,INT,Internal,,,,,,,,...,Cell_2_TH6_P3U30_protectTN,,,,,,,,,Obj3CSWI3\Pos\Oper\ctlVal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,RZA_4_RPN_pos,INT,Internal,,,,,,,,...,RZA_4_RPN2_Tapcon_RPN,,,,,,,,,ATCC1\TapPosBS\stVal
780,RZA_4_Comm_RPN,INT,Internal,,,,,,,,...,RZA_4_RPN2_Tapcon_RPN,,,,,,,,,ATCC1\TapChg\ctlVal
781,RZA_4_FailMotorRPN,DIGITAL,Internal,,,,,,,,...,RZA_4_RPN2_Tapcon_RPN,,,,,,,,,GGIO1\GPO1\stVal
782,RZA_4_Failure_RPN,DIGITAL,Internal,,,,,,,,...,RZA_4_RPN2_Tapcon_RPN,,,,,,,,,GGIO1\GPO2\stVal


In [30]:
ls = list(df1[df1['TYPE'] == 'DIGITAL']['NAME'].values)
ls = [f'Rand_dig("{nm}");' for nm in ls]
s5 = '\n'.join(ls)
print(s5)

Rand_dig("Cell_2_FailureTN");
Rand_dig("Cell_2_Start_Umax");
Rand_dig("Cell_2_ResetMPRZA");
Rand_dig("Cell_2_Start_Umin");
Rand_dig("Cell_2_Arc_protect");
Rand_dig("Cell_2_Kola_sw_off");
Rand_dig("Cell_2_Block_ZAZ");
Rand_dig("Cell_3_Arc_protect");
Rand_dig("Cell_3_Failure_kil");
Rand_dig("Cell_3_3U0_prot");
Rand_dig("Cell_3_ASDK_mode");
Rand_dig("Cell_3_Local_sw_on");
Rand_dig("Cell_3_Local_sw_off");
Rand_dig("Cell_3_PRVV_start");
Rand_dig("Cell_3_MSZ2");
Rand_dig("Cell_3_MSZ2_start");
Rand_dig("Cell_3_LZSH_start");
Rand_dig("Cell_3_MSZ1");
Rand_dig("Cell_3_MSZ1_start");
Rand_dig("Cell_3_ResetMPRZA");
Rand_dig("Cell_3_Pruz_ready");
Rand_dig("Cell_3_Kola_sw_off");
Rand_dig("Cell_3_Oper_sw_off");
Rand_dig("Cell_3_Block_ZAZ");
Rand_dig("Cell_3_Alarm_sw_off");
Rand_dig("Cell_4_Arc_protect");
Rand_dig("Cell_4_Failure_kil");
Rand_dig("Cell_4_3U0_prot");
Rand_dig("Cell_4_ASDK_mode");
Rand_dig("Cell_4_Local_sw_on");
Rand_dig("Cell_4_Local_sw_off");
Rand_dig("Cell_4_PRVV_start");
Rand_dig("Cel

In [31]:
ls = list(df1[df1['TYPE'] == 'INT']['NAME'].values)
ls = [f'Rand_dig3("{nm}");' for nm in ls]
s5 = '\n'.join(ls)
print(s5)

Rand_dig3("Cell_2_Oper_sw_off");
Rand_dig3("Cell_2_Command_VE");
Rand_dig3("Cell_2_Command_ZAZ");
Rand_dig3("Cell_2_Position_ZAZ");
Rand_dig3("Cell_2_Pos_L_R");
Rand_dig3("Cell_2_3U0_sw_off");
Rand_dig3("Cell_3_SW_pos");
Rand_dig3("Cell_3_Command_SW");
Rand_dig3("Cell_3_VE_pos");
Rand_dig3("Cell_3_Command_VE");
Rand_dig3("Cell_3_Command_ZAZ");
Rand_dig3("Cell_3_Block_VE");
Rand_dig3("Cell_3_Pos_L_R");
Rand_dig3("Cell_4_SW_pos");
Rand_dig3("Cell_4_Command_SW");
Rand_dig3("Cell_4_VE_pos");
Rand_dig3("Cell_4_Command_VE");
Rand_dig3("Cell_4_Command_ZAZ");
Rand_dig3("Cell_4_Block_VE");
Rand_dig3("Cell_4_Pos_L_R");
Rand_dig3("Cell_5_SW_pos");
Rand_dig3("Cell_5_Command_SW");
Rand_dig3("Cell_5_VE_pos");
Rand_dig3("Cell_5_Command_VE");
Rand_dig3("Cell_5_Command_ZAZ");
Rand_dig3("Cell_5_ZAZ_pos");
Rand_dig3("Cell_5_Block_VE");
Rand_dig3("Cell_5_Pos_L_R");
Rand_dig3("Cell_6_SW_pos");
Rand_dig3("Cell_6_Command_SW");
Rand_dig3("Cell_6_VE_pos");
Rand_dig3("Cell_6_Command_VE");
Rand_dig3("Cell_6_Comm

In [29]:
sr = df1[df1['ITEM'].str.contains(r'mag')]
ls = []
for i, tag in sr.iterrows():
    if '\\A\\' in tag['ITEM']:
        ls.append(f'Rand_analg("{tag.NAME}", 10, 9);')
    if '\\B\\' in tag['ITEM']:
        ls.append(f'Rand_analg("{tag.NAME}", 10, 9);')
    if '\\C\\' in tag['ITEM']:
        ls.append(f'Rand_analg("{tag.NAME}", 10, 9);')
    if '\\PPV\\' in tag['ITEM']:
        ls.append(f'Rand_analg_int("{tag.NAME}", 1000, 5500);')
    if '\\PhV\\' in tag['ITEM']:
        ls.append(f'Rand_analg_int("{tag.NAME}", 1000, 5500);')
    if '\\Hz\\' in tag['ITEM']:
        ls.append(f'Rand_analg("{tag.NAME}", 20, 40);')
s5 = '\n'.join(ls)
print(s5)

Rand_analg("Cell_2_F", 20, 40);
Rand_analg("Cell_2_uAB", 80, 200);
Rand_analg("Cell_2_uBC", 80, 200);
Rand_analg("Cell_2_uCA", 80, 200);
Rand_analg("Cell_2_u30", 80, 200);
Rand_analg("Cell_3_iA", 10, 9);
Rand_analg("Cell_3_iB", 10, 9);
Rand_analg("Cell_3_iC", 10, 9);
Rand_analg("Cell_3_i30", 10, 9);
Rand_analg("Cell_3_uAB", 80, 200);
Rand_analg("Cell_3_uBC", 80, 200);
Rand_analg("Cell_3_uAC", 80, 200);
Rand_analg("Cell_4_iA", 10, 9);
Rand_analg("Cell_4_iB", 10, 9);
Rand_analg("Cell_4_iC", 10, 9);
Rand_analg("Cell_4_i30", 10, 9);
Rand_analg("Cell_4_uAB", 80, 200);
Rand_analg("Cell_4_uBC", 80, 200);
Rand_analg("Cell_4_uAC", 80, 200);
Rand_analg("Cell_5_iA", 10, 9);
Rand_analg("Cell_5_iB", 10, 9);
Rand_analg("Cell_5_iC", 10, 9);
Rand_analg("Cell_5_uAB", 80, 200);
Rand_analg("Cell_5_uBC", 80, 200);
Rand_analg("Cell_5_uCA", 80, 200);
Rand_analg("Cell_5_u30", 80, 200);
Rand_analg("Cell_6_iA", 10, 9);
Rand_analg("Cell_6_iB", 10, 9);
Rand_analg("Cell_6_iC", 10, 9);
Rand_analg("Cell_6_i30", 10,

In [7]:
project = Project('Vostochnaya_v3')
df1 = project.base.base_tag.dataframe()
sr = df1[df1['ITEM'].str.contains(r'stVal|general|ctlVal', regex=True)]
ls = []
for tag in sr['NAME'].values:
    ls.append(f'Rand_dig("{tag}");')
s5 = '\n'.join(ls)
print(s5)

Rand_dig("Cell_2_TH6\VO3GGIO99\Ind\stVal");
Rand_dig("Cell_2_TH6\VO4GGIO100\Ind\stVal");
Rand_dig("Cell_2_TH6\Obj2CSWI2\Pos\stVal");
Rand_dig("Cell_2_TH6\Obj3CSWI3\Pos\Oper\ctlVal");
Rand_dig("Cell_2_TH6\local_Obj2CSWI2_Pos_stVal\");
Rand_dig("Cell_2_TH6\Obj2CSWI2\Pos\Oper\ctlVal");
Rand_dig("Cell_2_TH6\LLN0\Loc\stVal");
Rand_dig("Cell_2_TH6\Obj3CSWI3\Pos\stVal");
Rand_dig("Cell_2_TH6\ReLaGGIO141\Ind\stVal");
Rand_dig("Cell_2_TH6\VO5GGIO101\Ind\stVal");
Rand_dig("Cell_2_TH6\NSVPTOV1\Health\stVal");
Rand_dig("Cell_2_TH6\OV2PTOV4\Str\general");
Rand_dig("Cell_2_TH6\UV2PTUV2\Str\general");
Rand_dig("Cell_2_TH6\VO1GGIO97\Ind\stVal");
Rand_dig("Cell_3_LN6\VO3GGIO99\Ind\stVal");
Rand_dig("Cell_3_LN6\VO4GGIO100\Ind\stVal");
Rand_dig("Cell_3_LN6\Obj2CSWI2\Pos\stVal");
Rand_dig("Cell_3_LN6\Obj3CSWI3\Pos\Oper\ctlVal");
Rand_dig("Cell_3_LN6\local_Obj2CSWI2_Pos_stVal\");
Rand_dig("Cell_3_LN6\Obj2CSWI2\Pos\Oper\ctlVal");
Rand_dig("Cell_3_LN6\Obj3CSWI3\Pos\stVal");
Rand_dig("Cell_3_LN6\ReLaGGIO141\I

In [37]:
s1 = 'Cell_4_LN6_P3F30_line6'
match = re.search(r'(\S+)_(\S+)_(\S+)_(\S+)_(\S+)', s1)
x1 = match.group(1)
x2 = match.group(2)
x3 = match.group(3)

In [39]:
sr

,NAME,TYPE,UNIT,ADDR,RAW_ZERO,RAW_FULL,ENG_ZERO,ENG_FULL,ENG_UNITS,FORMAT,...,EQUIP,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,CUSTOM5,CUSTOM6,CUSTOM7,CUSTOM8,ITEM
0,Cell_2_F,REAL,Internal,,0.0,100.0,30.0,70.0,Hz,###.##,...,Cell_2_TH6_P3U30_protectTN,,,,,,,,,fMMXU8\Hz\mag\f
9,Cell_2_uAB,REAL,Internal,,0.0,100.0,200.0,280.0,Volts,###.##,...,Cell_2_TH6_P3U30_protectTN,,,,,,,,,U3ppMMXU5\PPV\phsAB\cVal\mag\f
10,Cell_2_uBC,REAL,Internal,,0.0,100.0,200.0,280.0,Volts,###.##,...,Cell_2_TH6_P3U30_protectTN,,,,,,,,,U3ppMMXU5\PPV\phsBC\cVal\mag\f
11,Cell_2_uCA,REAL,Internal,,0.0,100.0,200.0,280.0,Volts,###.##,...,Cell_2_TH6_P3U30_protectTN,,,,,,,,,U3ppMMXU5\PPV\phsCA\cVal\mag\f
12,Cell_2_u30,REAL,Internal,,0.0,100.0,200.0,280.0,Volts,###.##,...,Cell_2_TH6_P3U30_protectTN,,,,,,,,,UovMMXU1\PhV\neut\cVal\mag\f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720,RZA_4_iB,REAL,Internal,,0.0,100.0,10.0,20.0,Amps,###.##,...,RZA_4_T2_P3T32_mainT,,,,,,,,,I3pMMXU1\A\phsB\cVal\mag\f
721,RZA_4_iC,REAL,Internal,,0.0,100.0,10.0,20.0,Amps,###.##,...,RZA_4_T2_P3T32_mainT,,,,,,,,,I3pMMXU1\A\phsC\cVal\mag\f
765,RZA_5_iA,REAL,Internal,,0.0,100.0,10.0,20.0,Amps,###.##,...,RZA_5_T2_C264_reserveT,,,,,,,,,I3pMMXU1\A\phsA\cVal\mag\f
766,RZA_5_iB,REAL,Internal,,0.0,100.0,10.0,20.0,Amps,###.##,...,RZA_5_T2_C264_reserveT,,,,,,,,,I3pMMXU1\B\phsB\cVal\mag\f


In [3]:
project = Project('Vostochnaya_v3')
df1 = project.base.base_tag.dataframe()
# sr = df1[~((df1['NAME'].str.contains(r'\\t')) | (df1['NAME'].str.contains(r'\\q'))| (df1['NAME'].str.contains(r'\\T')))]
sr = df1[df1['ITEM'].str.contains(r'MMXU')]
sr = sr[~((sr['ITEM'].str.contains(r'\\t')) | (sr['ITEM'].str.contains(r'\\q')) | (sr['ITEM'].str.contains(r'\\T')))]

ds = {
    'TH6': 'tn',
    'LN6': 'ln',
    'IN6': 'in',
    'CB6': 'cb',
    'T1': 't1',
    'T2': 't2',
    'CB35': 'cb',
    'RPN1': 'rpn1',
    'RPN2': 'rpn2',
}

ls4 = []
cols = []
for i, tag in sr.iterrows():
    # match = re.search(r'(\S+)_(\S+)_(\S+?)\\', tag)
    match = re.search(r'(\S+)_(\S+)_(\S+)_(\S+)_(\S+)', tag['EQUIP'])
    match1 = re.search(r'(\S+)\\(\S+$)', tag['ITEM'])
    x1 = match.group(1)
    x2 = match.group(2)
    x3 = match.group(3)
    x4 = match1.group(2)

    shkaf_name = 'c'
    if x1 == 'RZA': shkaf_name = 'r'

    val = ''
    if 'phsAB\\' in tag['ITEM']: val = 'uab'
    if 'phsBC\\' in tag['ITEM']: val = 'ubc'
    if 'phsCA\\' in tag['ITEM']: val = 'uca'
    if 'phsA\\' in tag['ITEM']: val = 'ia'
    if 'phsB\\' in tag['ITEM']: val = 'ib'
    if 'phsC\\' in tag['ITEM']: val = 'ic'
    if 'A\\neut' in tag['ITEM']: val = '3io'
    if 'PhV\\neut' in tag['ITEM']: val = '3uo'
    if 'Hz' in tag['ITEM']: val = 'hz'

    col_base = f'{shkaf_name}{x2}_{ds[x3]}_{val}_'
    col = f'{shkaf_name}{x2}_{ds[x3]}_{val}_{x4}'
    cols.append(col)

    s1 = rf'MapValueSet(sm1, "{col}",  RealToStr({tag.NAME}, 5, 2, "."));'
    # s11 = rf'MapValueSet(sm1, "{col_base}t", TagGetValue("{tag.NAME}.Field", 5));'
    s11 = rf'MapValueSet(sm1, "{col_base}t", getTimeTag("{tag.NAME}.Field"));'
    s12 = rf'MapValueSet(sm1, "{col_base}q", getQualityTag("{tag.NAME}.Field"));'

    ls4.append(s1)
    ls4.append(s11)
    ls4.append(s12)
s5 = '\n'.join(ls4)
print(s5)

MapValueSet(sm1, "c2_tn_hz_f",  RealToStr(Cell_2_F, 5, 2, "."));
MapValueSet(sm1, "c2_tn_hz_t", getTimeTag("Cell_2_F.Field"));
MapValueSet(sm1, "c2_tn_hz_q", getQualityTag("Cell_2_F.Field"));
MapValueSet(sm1, "c2_tn_uab_f",  RealToStr(Cell_2_uAB, 5, 2, "."));
MapValueSet(sm1, "c2_tn_uab_t", getTimeTag("Cell_2_uAB.Field"));
MapValueSet(sm1, "c2_tn_uab_q", getQualityTag("Cell_2_uAB.Field"));
MapValueSet(sm1, "c2_tn_ubc_f",  RealToStr(Cell_2_uBC, 5, 2, "."));
MapValueSet(sm1, "c2_tn_ubc_t", getTimeTag("Cell_2_uBC.Field"));
MapValueSet(sm1, "c2_tn_ubc_q", getQualityTag("Cell_2_uBC.Field"));
MapValueSet(sm1, "c2_tn_uca_f",  RealToStr(Cell_2_uCA, 5, 2, "."));
MapValueSet(sm1, "c2_tn_uca_t", getTimeTag("Cell_2_uCA.Field"));
MapValueSet(sm1, "c2_tn_uca_q", getQualityTag("Cell_2_uCA.Field"));
MapValueSet(sm1, "c2_tn_3uo_f",  RealToStr(Cell_2_u30, 5, 2, "."));
MapValueSet(sm1, "c2_tn_3uo_t", getTimeTag("Cell_2_u30.Field"));
MapValueSet(sm1, "c2_tn_3uo_q", getQualityTag("Cell_2_u30.Field"));
MapV

In [49]:
match = re.search(r'(\S+)_(\S+)_(\S+?)\\', s1)

In [50]:
print(match.group(1))
print(match.group(2))
print(match.group(3))

Cell
2
TH6


In [5]:
project = Project('test61850_2')
df1 = project.base.base_tag.dataframe()
sr = df1[df1['NAME'].str.contains(r'MMXU')]

ds = {
    'TH6': 'tn',
    'LN6': 'ln',
    'IN6': 'in',
    'CB6': 'cb',
    'T1': 't1',
    'T2': 't2',
    'CB35': 'cb',
}

ls = []
ls1 = []
ls2 = []
ls3 = []
cols = []
for tag in sr['NAME'].values:
    # match = re.search(r'(\S+)_(\S+)_(\S+?)\\', tag)
    match = re.search(r'(\S+)_(\S+)_(\S+?)\\', tag)
    match1 = re.search(r'(\S+)\\(\S+$)', tag)
    x1 = match.group(1)
    x2 = match.group(2)
    x3 = match.group(3)
    x4 = match1.group(2)

    shkaf_name = 'c'
    if x1 == 'RZA': shkaf_name = 'r'

    val = ''
    if 'phsAB\\' in tag: val = 'uab'
    if 'phsBC\\' in tag: val = 'ubc'
    if 'phsCA\\' in tag: val = 'uca'
    if 'phsA\\' in tag: val = 'ia'
    if 'phsB\\' in tag: val = 'ib'
    if 'phsC\\' in tag: val = 'ic'
    if 'A\\neut' in tag: val = '3io'
    if 'PhV\\neut' in tag: val = '3uo'
    if 'Hz' in tag: val = 'hz'

    col = f'{shkaf_name}{x2}_{ds[x3]}_{val}_{x4}'
    cols.append(col)
    s1 = rf'MapValueSet(sm1, "{col}", {tag});'
    s2 = f'SQLAppend(hSQL, MapValueGet(smap1 , "{col}") + ",");'
    if x4 in ['t', 'q']:
        s2 = f'''SQLAppend(hSQL, "'" + MapValueGet(smap1 , "{col}") + "',");'''
    s3 = f'SQLAppend(hSQL, "{col},");'
    ls.append(s1)
    ls1.append(s2)
    ls2.append(s3)

    typ = 'real'
    if x4.lower() == 't': typ = 'datetime'
    if x4 == 'q': typ = 'nvarchar(30)'
    ls3.append(f'{col} {typ},')
ls
r'MapValueSet(sm1, "c3l_ia", Cell_3_LN6_P3F30_line6.I3pMMXU1\A\phsA\cVal\mag\f);'

'MapValueSet(sm1, "c3l_ia", Cell_3_LN6_P3F30_line6.I3pMMXU1\\A\\phsA\\cVal\\mag\\f);'

In [6]:
s5 = '\n'.join(ls)
print(s5)

MapValueSet(sm1, "c2_tn_hz_f", Cell_2_TH6\fMMXU8\Hz\mag\f);
MapValueSet(sm1, "c2_tn_hz_t", Cell_2_TH6\fMMXU8\Hz\t);
MapValueSet(sm1, "c2_tn_hz_q", Cell_2_TH6\fMMXU8\Hz\q);
MapValueSet(sm1, "c2_tn_uab_f", Cell_2_TH6\U3ppMMXU5\PPV\phsAB\cVal\mag\f);
MapValueSet(sm1, "c2_tn_uab_t", Cell_2_TH6\U3ppMMXU5\PPV\phsAB\t);
MapValueSet(sm1, "c2_tn_uab_q", Cell_2_TH6\U3ppMMXU5\PPV\phsAB\q);
MapValueSet(sm1, "c2_tn_ubc_f", Cell_2_TH6\U3ppMMXU5\PPV\phsBC\cVal\mag\f);
MapValueSet(sm1, "c2_tn_ubc_t", Cell_2_TH6\U3ppMMXU5\PPV\phsBC\t);
MapValueSet(sm1, "c2_tn_ubc_q", Cell_2_TH6\U3ppMMXU5\PPV\phsBC\q);
MapValueSet(sm1, "c2_tn_uca_f", Cell_2_TH6\U3ppMMXU5\PPV\phsCA\cVal\mag\f);
MapValueSet(sm1, "c2_tn_uca_t", Cell_2_TH6\U3ppMMXU5\PPV\phsCA\t);
MapValueSet(sm1, "c2_tn_uca_q", Cell_2_TH6\U3ppMMXU5\PPV\phsCA\q);
MapValueSet(sm1, "c2_tn_3uo_f", Cell_2_TH6\UovMMXU1\PhV\neut\cVal\mag\f);
MapValueSet(sm1, "c2_tn_3uo_t", Cell_2_TH6\UovMMXU1\PhV\neut\t);
MapValueSet(sm1, "c2_tn_3uo_q", Cell_2_TH6\UovMMXU1\PhV\neu

In [24]:
sr = df1[df1['NAME'].str.contains(r'stVal|general|ctlVal', regex=True)]


        subsHandle = TagSubscribe("Cell_2_TH6\VO3GGIO99\Ind\stVal");
	    SubscriptionAddCallback(subsHandle, "OnValueChanged");
    

        subsHandle = TagSubscribe("Cell_2_TH6\VO4GGIO100\Ind\stVal");
	    SubscriptionAddCallback(subsHandle, "OnValueChanged");
    

        subsHandle = TagSubscribe("Cell_2_TH6\Obj2CSWI2\Pos\stVal");
	    SubscriptionAddCallback(subsHandle, "OnValueChanged");
    

        subsHandle = TagSubscribe("Cell_2_TH6\Obj3CSWI3\Pos\Oper\ctlVal");
	    SubscriptionAddCallback(subsHandle, "OnValueChanged");
    

        subsHandle = TagSubscribe("Cell_2_TH6\local_Obj2CSWI2_Pos_stVal\");
	    SubscriptionAddCallback(subsHandle, "OnValueChanged");
    

        subsHandle = TagSubscribe("Cell_2_TH6\Obj2CSWI2\Pos\Oper\ctlVal");
	    SubscriptionAddCallback(subsHandle, "OnValueChanged");
    

        subsHandle = TagSubscribe("Cell_2_TH6\LLN0\Loc\stVal");
	    SubscriptionAddCallback(subsHandle, "OnValueChanged");
    

        subsHandle = TagSubscribe("Cell

In [26]:
ls = []

for tag in sr['NAME'].values:
    # s1 = f'''
    #     subsHandle = TagSubscribe("{tag}");
    #     Sub
    #     scriptionAddCallback(subsHandle, "OnValueChanged");
    # ''' 
    print(f'subsHandle = TagSubscribe("{tag}");')
    print('SubscriptionAddCallback(subsHandle, "OnValueChanged");')
    # ls.append(s1)

# s5 = '\n'.join(ls)
# print(s5)

subsHandle = TagSubscribe("Cell_2_TH6\VO3GGIO99\Ind\stVal");
SubscriptionAddCallback(subsHandle, "OnValueChanged");
subsHandle = TagSubscribe("Cell_2_TH6\VO4GGIO100\Ind\stVal");
SubscriptionAddCallback(subsHandle, "OnValueChanged");
subsHandle = TagSubscribe("Cell_2_TH6\Obj2CSWI2\Pos\stVal");
SubscriptionAddCallback(subsHandle, "OnValueChanged");
subsHandle = TagSubscribe("Cell_2_TH6\Obj3CSWI3\Pos\Oper\ctlVal");
SubscriptionAddCallback(subsHandle, "OnValueChanged");
subsHandle = TagSubscribe("Cell_2_TH6\local_Obj2CSWI2_Pos_stVal\");
SubscriptionAddCallback(subsHandle, "OnValueChanged");
subsHandle = TagSubscribe("Cell_2_TH6\Obj2CSWI2\Pos\Oper\ctlVal");
SubscriptionAddCallback(subsHandle, "OnValueChanged");
subsHandle = TagSubscribe("Cell_2_TH6\LLN0\Loc\stVal");
SubscriptionAddCallback(subsHandle, "OnValueChanged");
subsHandle = TagSubscribe("Cell_2_TH6\Obj3CSWI3\Pos\stVal");
SubscriptionAddCallback(subsHandle, "OnValueChanged");
subsHandle = TagSubscribe("Cell_2_TH6\ReLaGGIO141\Ind\stV

In [ ]:
project = Project('Vostochnaya_v3')
df1 = project.base.base_tag.dataframe()
# sr = df1[~((df1['NAME'].str.contains(r'\\t')) | (df1['NAME'].str.contains(r'\\q'))| (df1['NAME'].str.contains(r'\\T')))]
sr = df1[df1['NAME'].str.contains(r'MMXU')]
sr = sr[~((sr['NAME'].str.contains(r'\\t')) | (sr['NAME'].str.contains(r'\\q')) | (sr['NAME'].str.contains(r'\\T')))]

ds = {
    'TH6': 'tn',
    'LN6': 'ln',
    'IN6': 'in',
    'CB6': 'cb',
    'T1': 't1',
    'T2': 't2',
    'CB35': 'cb',
}

ls4 = []
cols = []
for tag in sr['NAME'].values:
    # match = re.search(r'(\S+)_(\S+)_(\S+?)\\', tag)
    match = re.search(r'(\S+)_(\S+)_(\S+?)\\', tag)
    match1 = re.search(r'(\S+)\\(\S+$)', tag)
    x1 = match.group(1)
    x2 = match.group(2)
    x3 = match.group(3)
    x4 = match1.group(2)

    shkaf_name = 'c'
    if x1 == 'RZA': shkaf_name = 'r'

    val = ''
    if 'phsAB\\' in tag: val = 'uab'
    if 'phsBC\\' in tag: val = 'ubc'
    if 'phsCA\\' in tag: val = 'uca'
    if 'phsA\\' in tag: val = 'ia'
    if 'phsB\\' in tag: val = 'ib'
    if 'phsC\\' in tag: val = 'ic'
    if 'A\\neut' in tag: val = '3io'
    if 'PhV\\neut' in tag: val = '3uo'
    if 'Hz' in tag: val = 'hz'

    col_base = f'{shkaf_name}{x2}_{ds[x3]}_{val}_'
    col = f'{shkaf_name}{x2}_{ds[x3]}_{val}_{x4}'
    cols.append(col)

    s1 = rf'MapValueSet(sm1, "{col}", {tag});'
    s11 = rf'MapValueSet(sm1, "{col_base}t", TagGetValue("{tag}.Field", 5));'
    s12 = rf'MapValueSet(sm1, "{col_base}q", TagGetValue("{tag}.Field", 3));'

    ls4.append(s1)
    ls4.append(s11)
    ls4.append(s12)


In [ ]:
s5 = '\n'.join(ls4)
print(s5)

In [80]:
project = Project('test61850_2')
df1 = project.base.base_tag.dataframe()

ds = {
    'TH6': 'tn',
    'LN6': 'ln',
    'IN6': 'in',
    'CB6': 'cb',
    'T1': 't1'
}

sr = df1[df1['NAME'].str.contains(r'mag')]

ls = []
ls1 = []
ls2 = []
ls3 = []
cols = []
for tag in sr['NAME'].values:
    match = re.search(r'(\S+)_(\S+)_(\S+?)\\', tag)
    x1 = match.group(1)
    x2 = match.group(2)
    x3 = match.group(3)

    if x1 == 'Cell' or x1 == 'RZA':
        val = ''
        if 'phsAB\\' in tag: val = 'uab'
        if 'phsBC\\' in tag: val = 'ubc'
        if 'phsCA\\' in tag: val = 'uca'
        if 'phsA\\' in tag: val = 'ia'
        if 'phsB\\' in tag: val = 'ib'
        if 'phsC\\' in tag: val = 'ic'
        if 'A\\neut' in tag: val = '3io'
        if 'PhV\\neut' in tag: val = '3uo'
        if 'Hz' in tag: val = 'f'

        col = f'c{x2}_{ds[x3]}_{val}'
        cols.append(col)
        s1 = rf'MapValueSet(sm1, "{col}", {tag});'
        s2 = f'SQLAppend(hSQL, MapValueGet(smap1 , "{col}") + ",");'
        s3 = f'SQLAppend(hSQL, "{col},");'
        ls.append(s1)
        ls1.append(s2)
        ls2.append(s3)

        ls3.append(f'{col} real,')
        ls3.append(f'{col}_t datetime,')
        ls3.append(f'{col}_q NVARCHAR(20),')
ls
r'MapValueSet(sm1, "c3l_ia", Cell_3_LN6_P3F30_line6.I3pMMXU1\A\phsA\cVal\mag\f);'

KeyError: 'T1'

In [73]:
s5 = '\n'.join(ls)
print(s5)

MapValueSet(sm1, "c2_tn_f", Cell_2_TH6\fMMXU8\Hz\mag\f);
MapValueSet(sm1, "c2_tn_uab", Cell_2_TH6\U3ppMMXU5\PPV\phsAB\cVal\mag\f);
MapValueSet(sm1, "c2_tn_ubc", Cell_2_TH6\U3ppMMXU5\PPV\phsBC\cVal\mag\f);
MapValueSet(sm1, "c2_tn_uca", Cell_2_TH6\U3ppMMXU5\PPV\phsCA\cVal\mag\f);
MapValueSet(sm1, "c2_tn_3uo", Cell_2_TH6\UovMMXU1\PhV\neut\cVal\mag\f);
MapValueSet(sm1, "c3_ln_ia", Cell_3_LN6\I3pMMXU1\A\phsA\cVal\mag\f);
MapValueSet(sm1, "c3_ln_ib", Cell_3_LN6\I3pMMXU1\A\phsB\cVal\mag\f);
MapValueSet(sm1, "c3_ln_ic", Cell_3_LN6\I3pMMXU1\A\phsC\cVal\mag\f);
MapValueSet(sm1, "c3_ln_uab", Cell_3_LN6\U3ppMMXU5\PPV\phsAB\cVal\mag\f);
MapValueSet(sm1, "c3_ln_ubc", Cell_3_LN6\U3ppMMXU5\PPV\phsBC\cVal\mag\f);
MapValueSet(sm1, "c3_ln_uca", Cell_3_LN6\U3ppMMXU5\PPV\phsCA\cVal\mag\f);
MapValueSet(sm1, "c3_ln_3io", Cell_3_LN6\Io1MMXU11\A\neut\cVal\mag\f);
MapValueSet(sm1, "c4_ln_ia", Cell_4_LN6\I3pMMXU1\A\phsA\cVal\mag\f);
MapValueSet(sm1, "c4_ln_ib", Cell_4_LN6\I3pMMXU1\A\phsB\cVal\mag\f);
MapValueSet

In [79]:
print('\n'.join(ls3))

c2_tn_f real,
c2_tn_f_t datetime,
c2_tn_f_q NVARCHAR(20),
c2_tn_uab real,
c2_tn_uab_t datetime,
c2_tn_uab_q NVARCHAR(20),
c2_tn_ubc real,
c2_tn_ubc_t datetime,
c2_tn_ubc_q NVARCHAR(20),
c2_tn_uca real,
c2_tn_uca_t datetime,
c2_tn_uca_q NVARCHAR(20),
c2_tn_3uo real,
c2_tn_3uo_t datetime,
c2_tn_3uo_q NVARCHAR(20),
c3_ln_ia real,
c3_ln_ia_t datetime,
c3_ln_ia_q NVARCHAR(20),
c3_ln_ib real,
c3_ln_ib_t datetime,
c3_ln_ib_q NVARCHAR(20),
c3_ln_ic real,
c3_ln_ic_t datetime,
c3_ln_ic_q NVARCHAR(20),
c3_ln_uab real,
c3_ln_uab_t datetime,
c3_ln_uab_q NVARCHAR(20),
c3_ln_ubc real,
c3_ln_ubc_t datetime,
c3_ln_ubc_q NVARCHAR(20),
c3_ln_uca real,
c3_ln_uca_t datetime,
c3_ln_uca_q NVARCHAR(20),
c3_ln_3io real,
c3_ln_3io_t datetime,
c3_ln_3io_q NVARCHAR(20),
c4_ln_ia real,
c4_ln_ia_t datetime,
c4_ln_ia_q NVARCHAR(20),
c4_ln_ib real,
c4_ln_ib_t datetime,
c4_ln_ib_q NVARCHAR(20),
c4_ln_ic real,
c4_ln_ic_t datetime,
c4_ln_ic_q NVARCHAR(20),
c4_ln_uab real,
c4_ln_uab_t datetime,
c4_ln_uab_q NVARCHAR(20),


In [ ]:
file_path = r'c:\libCitectScada\libDevice.xlsx'
xll = pd.ExcelFile(file_path)
xll.sheet_names
for sheet_name in xll.sheet_names:
    df1 = pd.read_excel(file_path, sheet_name=sheet_name)
    ls = df1.values.tolist()
    ls_new = []
    for ls1 in ls:
        ls_new.append(ls1)
        s1 = ls1[-1]
        match = re.search(r'(час)', s1)
        if match:
            ls2 = [ls1[0]]
            s2 = ls1[1]
            s2 = s2.replace('/t', '/q')
            s2 = s2.replace('/T', '/q')
            ls2.append(s2)
            ls2.append(ls1[2])
            ls2.append(ls1[3])
            s3 = ls1[-1]
            ls2.append(s3.replace('(час)', '(якість)'))
            ls_new.append(ls2)

    ls_new

    df2 = pd.DataFrame(ls_new, columns=df1.columns)

    file_path1 = r'c:\libCitectScada\libDevice1.xlsx'
    with ExcelWriter(file_path1, mode="a" if os.path.exists(file_path1) else "w") as writer:
        df2.to_excel(writer, sheet_name=sheet_name, index=False)



In [4]:
project = Project('test61850_2')
project.copy_libfile(['libDevice.xlsx', 'setProject.xlsx'])
project.create_dbf_from_xls([('boards', 'NAME'), ('ports', 'NAME')])

project.create_eqtype()
project.create_io()
# project.create_equip()
project.create_equip(isCreateTag=True)

In [48]:
df_tag = pd.read_excel(project.config_setProject_xls, sheet_name='Tag')
project.create_tag(df_tag)

In [3]:
project = Project('test61850_2')
project.base.base_io.clear_dbf()
project.base.base_equip.clear_dbf()
project.base.base_tag.clear_dbf()

In [106]:
project = Project('tmp1')
df_tag = pd.read_excel(project.config_xls, sheet_name='Tag')
# project.create_tag_simple(df_tag)
project.create_tag(df_tag, replace=True)

In [3]:
project = Project('Proj16850_17112021')
df2 = project.base.base_tag.dataframe()
df2

,NAME,TYPE,UNIT,ADDR,RAW_ZERO,RAW_FULL,ENG_ZERO,ENG_FULL,ENG_UNITS,FORMAT,...,EQUIP,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,CUSTOM5,CUSTOM6,CUSTOM7,CUSTOM8,ITEM
0,Cell_2\Obj3CSWI3\Pos\Oper\ctlVal,DIGITAL,C264,Relay/Obj3CSWI3$CO$Pos$Oper$ctlVal,,,,,,,...,Cell_2_P3U30,,,,,,,,,Obj3CSWI3\Pos\Oper\ctlVal
1,Cell_2\Obj2CSWI2\Pos\stVal,STRING,C264,Relay/Obj2CSWI2$ST$Pos$stVal,,,,,,,...,Cell_2_P3U30,,,,,,,,,Obj2CSWI2\Pos\stVal
2,Cell_2\Obj2CSWI2\Pos\t,STRING,C264,Relay/Obj2CSWI2$ST$Pos$t,,,,,,,...,Cell_2_P3U30,,,,,,,,,Obj2CSWI2\Pos\t
3,Cell_2\local_Obj2CSWI2_Pos_stVal,INT,Internal,,,,,,,,...,Cell_2_P3U30,,,,,,,,,local_Obj2CSWI2_Pos_stVal
4,Cell_2\Obj2CSWI2\Pos\Oper\ctlVal,DIGITAL,C264,Relay/Obj2CSWI2$CO$Pos$Oper$ctlVal,,,,,,,...,Cell_2_P3U30,,,,,,,,,Obj2CSWI2\Pos\Oper\ctlVal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,RZA5\U3ppMMXU5\PPV\phsBC\t,STRING,C264,RZA5/U3ppMMXU5$MX$PPV$phsBC$t,,,,,,,...,RZA5_C264,,,,,,,,,U3ppMMXU5\PPV\phsBC\t
165,RZA5\U3ppMMXU5\PPV\phsCA\cVal\mag,STRING,C264,RZA5/U3ppMMXU5$MX$PPV$phsCA$cVal$mag,,,,,,,...,RZA5_C264,,,,,,,,,U3ppMMXU5\PPV\phsCA\cVal\mag
166,RZA5\U3ppMMXU5\PPV\phsCA\t,STRING,C264,RZA5/U3ppMMXU5$MX$PPV$phsCA$t,,,,,,,...,RZA5_C264,,,,,,,,,U3ppMMXU5\PPV\phsCA\t
167,RZA5\Io1MMXU11\A\neut\cVal\mag,STRING,C264,RZA5/Io1MMXU11$MX$A$neut$cVal$mag,,,,,,,...,RZA5_C264,,,,,,,,,Io1MMXU11\A\neut\cVal\mag


In [95]:
ls = [1, 2, 3, 4]
ls.append(5)
ls

[1, 2, 3, 4, 5]

In [65]:
df1 = project.base.base_tag.dataframe()
df1

,NAME,TYPE,UNIT,ADDR,RAW_ZERO,RAW_FULL,ENG_ZERO,ENG_FULL,ENG_UNITS,FORMAT,...,EQUIP,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,CUSTOM5,CUSTOM6,CUSTOM7,CUSTOM8,ITEM
0,i1,INT,Internal,,,,,,,,...,,,,,,,,,,
1,d1,DIGITAL,Internal,,,,,,,,...,,,,,,,,,,
2,r1,REAL,Internal,,,,,,,,...,,,,,,,,,,
3,s1,STRING,Internal,,,,,,,,...,,,,,,,,,,
4,d2,DIGITAL,Internal,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199,Cell_21\U3ppMMXU5\PPV\phsBC\t,STRING,PLC_C264_TU,Cell_21/U3ppMMXU5$MX\PPV$phsBC$t,,,,,,,...,Cell_21_TH_6kV_P3U30,,,,,,,,,U3ppMMXU5\PPV\phsBC\t
1200,Cell_21\U3ppMMXU5\PPV\phsCA\cVal,STRING,PLC_C264_TU,Cell_21/U3ppMMXU5$MX\PPV$phsCA$cVal,,,,,,,...,Cell_21_TH_6kV_P3U30,,,,,,,,,U3ppMMXU5\PPV\phsCA\cVal
1201,Cell_21\U3ppMMXU5\PPV\phsCA\t,STRING,PLC_C264_TU,Cell_21/U3ppMMXU5$MX\PPV$phsCA$t,,,,,,,...,Cell_21_TH_6kV_P3U30,,,,,,,,,U3ppMMXU5\PPV\phsCA\t
1202,Cell_21\UovMMXU1\PhV\neut\cVal,STRING,PLC_C264_TU,Cell_21/UovMMXU1$MX\PhV$neut$cVal,,,,,,,...,Cell_21_TH_6kV_P3U30,,,,,,,,,UovMMXU1\PhV\neut\cVal


In [90]:
values = {'NAME': '454588',
          'TYPE': '',
          'UNIT': '',
          'ADDR': "",
          'RAW_ZERO': '',
          'RAW_FULL': '',
          'ENG_ZERO': '',
          'ENG_FULL': '',
          'ENG_UNITS': '',
          'FORMAT': '',
          'COMMENT': '',
          'EDITCODE': '',
          'LINKED': '',
          'OID': '',
          'REF1': '',
          'REF2': '',
          'DEADBAND': '',
          'CUSTOM': '',
          'TAGGENLINK': '',
          'CLUSTER': '',
          'HISTORIAN': 'TRUE',
          'EQUIP': '',
          'CUSTOM1': '',
          'CUSTOM2': '',
          'CUSTOM3': '',
          'CUSTOM4': '',
          'CUSTOM5': '',
          'CUSTOM6': '',
          'CUSTOM7': '',
          'CUSTOM8': '',
          'ITEM': ''}

# pd.concat([df1, pd.DataFrame([values])], ignore_index=True)


In [91]:
df3 = pd.DataFrame()
df3

""


In [92]:
pd.concat([df3, pd.DataFrame([values])], ignore_index=True)

,NAME,TYPE,UNIT,ADDR,RAW_ZERO,RAW_FULL,ENG_ZERO,ENG_FULL,ENG_UNITS,FORMAT,...,EQUIP,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,CUSTOM5,CUSTOM6,CUSTOM7,CUSTOM8,ITEM
0,454588,,,,,,,,,,...,,,,,,,,,,


In [54]:
df1.loc[df1['NAME'] == 'd5', :] = list(values.values())

In [45]:
list(values.values())

['454588',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'TRUE',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [55]:
df1

,NAME,TYPE,UNIT,ADDR,RAW_ZERO,RAW_FULL,ENG_ZERO,ENG_FULL,ENG_UNITS,FORMAT,...,EQUIP,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,CUSTOM5,CUSTOM6,CUSTOM7,CUSTOM8,ITEM
0,i1,INT,Internal,,,,,,,,...,,,,,,,,,,
1,d1,DIGITAL,Internal,,,,,,,,...,,,,,,,,,,
2,r1,REAL,Internal,,,,,,,,...,,,,,,,,,,
3,s1,STRING,Internal,,,,,,,,...,,,,,,,,,,
4,d2,DIGITAL,Internal,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199,Cell_21\U3ppMMXU5\PPV\phsBC\t,STRING,PLC_C264_TU,Cell_21/U3ppMMXU5$MX\PPV$phsBC$t,,,,,,,...,Cell_21_TH_6kV_P3U30,,,,,,,,,U3ppMMXU5\PPV\phsBC\t
1200,Cell_21\U3ppMMXU5\PPV\phsCA\cVal,STRING,PLC_C264_TU,Cell_21/U3ppMMXU5$MX\PPV$phsCA$cVal,,,,,,,...,Cell_21_TH_6kV_P3U30,,,,,,,,,U3ppMMXU5\PPV\phsCA\cVal
1201,Cell_21\U3ppMMXU5\PPV\phsCA\t,STRING,PLC_C264_TU,Cell_21/U3ppMMXU5$MX\PPV$phsCA$t,,,,,,,...,Cell_21_TH_6kV_P3U30,,,,,,,,,U3ppMMXU5\PPV\phsCA\t
1202,Cell_21\UovMMXU1\PhV\neut\cVal,STRING,PLC_C264_TU,Cell_21/UovMMXU1$MX\PhV$neut$cVal,,,,,,,...,Cell_21_TH_6kV_P3U30,,,,,,,,,UovMMXU1\PhV\neut\cVal


In [229]:
pd.DataFrame([values])

,NAME,TYPE,UNIT,ADDR,RAW_ZERO,RAW_FULL,ENG_ZERO,ENG_FULL,ENG_UNITS,FORMAT,...,EQUIP,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,CUSTOM5,CUSTOM6,CUSTOM7,CUSTOM8,ITEM
0,454588,,,,,,,,,,...,,,,,,,,,,


In [205]:
df1 = project.base.base_tag.dataframe()
df1

,NAME,TYPE,UNIT,ADDR,RAW_ZERO,RAW_FULL,ENG_ZERO,ENG_FULL,ENG_UNITS,FORMAT,...,EQUIP,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,CUSTOM5,CUSTOM6,CUSTOM7,CUSTOM8,ITEM
0,Cell_2\Obj3CSWI3\Pos\Oper\ctlVal,DIGITAL,PLC_C264_TU,Cell_2/Obj3CSWI3$CO\Pos$Oper$ctlVal,,,,,,,...,Cell_2_TH_6kV_P3U30,,,,,,,,,Obj3CSWI3\Pos\Oper\ctlVal
1,Cell_2\Obj2CSWI2\Pos\stVal,STRING,PLC_C264_TU,Cell_2/Obj2CSWI2$ST\Pos$stVal,,,,,,,...,Cell_2_TH_6kV_P3U30,,,,,,,,,Obj2CSWI2\Pos\stVal
2,Cell_2\Obj2CSWI2\Pos\t,STRING,PLC_C264_TU,Cell_2/Obj2CSWI2$ST\Pos$t,,,,,,,...,Cell_2_TH_6kV_P3U30,,,,,,,,,Obj2CSWI2\Pos\t
3,Cell_2\local_Obj2CSWI2\Pos\stVal,INT,PLC_C264_TU,Cell_2/local_Obj2CSWI2$ST\Pos$stVal,,,,,,,...,Cell_2_TH_6kV_P3U30,,,,,,,,,local_Obj2CSWI2\Pos\stVal
4,Cell_2\Obj2CSWI2\Pos\Oper\ctlVal,DIGITAL,PLC_C264_TU,Cell_2/Obj2CSWI2$CO\Pos$Oper$ctlVal,,,,,,,...,Cell_2_TH_6kV_P3U30,,,,,,,,,Obj2CSWI2\Pos\Oper\ctlVal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199,s1,STRING,Internal,,,,,,,,...,,,,,,,,,,
1200,d2,DIGITAL,Internal,,,,,,,,...,,,,,,,,,,
1201,d3,DIGITAL,Internal,,,,,,,,...,,,,,,,,,,
1202,d4,DIGITAL,Internal,,,,,,,,...,,,,,,,,,,


In [208]:
df1[df1['NAME'] == 'd5'].loc[0, 'ADDR'] = '112'

C:\Users\Alex\AppData\Local\Temp/ipykernel_8036/1748805692.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[df1['NAME']=='d5'].loc[0, 'ADDR'] = '112'


In [222]:
df1['ADDR'].where(~(df1['NAME'] == 'd5'), other='1', inplace=True)

In [223]:
df1.iloc[-1]

NAME                d5
TYPE           DIGITAL
UNIT          Internal
ADDR                 1
RAW_ZERO              
RAW_FULL              
ENG_ZERO              
ENG_FULL              
ENG_UNITS             
FORMAT                
COMMENT               
EDITCODE              
LINKED                
OID                   
REF1                  
REF2                  
DEADBAND              
CUSTOM                
TAGGENLINK            
CLUSTER       Cluster1
HISTORIAN         TRUE
EQUIP                 
CUSTOM1               
CUSTOM2               
CUSTOM3               
CUSTOM4               
CUSTOM5               
CUSTOM6               
CUSTOM7               
CUSTOM8               
ITEM                  
Name: 1203, dtype: object

In [224]:
df1

,NAME,TYPE,UNIT,ADDR,RAW_ZERO,RAW_FULL,ENG_ZERO,ENG_FULL,ENG_UNITS,FORMAT,...,EQUIP,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,CUSTOM5,CUSTOM6,CUSTOM7,CUSTOM8,ITEM
0,Cell_2\Obj3CSWI3\Pos\Oper\ctlVal,DIGITAL,PLC_C264_TU,0,,,,,,,...,Cell_2_TH_6kV_P3U30,,,,,,,,,Obj3CSWI3\Pos\Oper\ctlVal
1,Cell_2\Obj2CSWI2\Pos\stVal,STRING,PLC_C264_TU,0,,,,,,,...,Cell_2_TH_6kV_P3U30,,,,,,,,,Obj2CSWI2\Pos\stVal
2,Cell_2\Obj2CSWI2\Pos\t,STRING,PLC_C264_TU,0,,,,,,,...,Cell_2_TH_6kV_P3U30,,,,,,,,,Obj2CSWI2\Pos\t
3,Cell_2\local_Obj2CSWI2\Pos\stVal,INT,PLC_C264_TU,0,,,,,,,...,Cell_2_TH_6kV_P3U30,,,,,,,,,local_Obj2CSWI2\Pos\stVal
4,Cell_2\Obj2CSWI2\Pos\Oper\ctlVal,DIGITAL,PLC_C264_TU,0,,,,,,,...,Cell_2_TH_6kV_P3U30,,,,,,,,,Obj2CSWI2\Pos\Oper\ctlVal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199,s1,STRING,Internal,0,,,,,,,...,,,,,,,,,,
1200,d2,DIGITAL,Internal,0,,,,,,,...,,,,,,,,,,
1201,d3,DIGITAL,Internal,0,,,,,,,...,,,,,,,,,,
1202,d4,DIGITAL,Internal,0,,,,,,,...,,,,,,,,,,


In [163]:
df1[df1['NAME'] == 'd5']

,NAME,TYPE,UNIT,ADDR,RAW_ZERO,RAW_FULL,ENG_ZERO,ENG_FULL,ENG_UNITS,FORMAT,...,EQUIP,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,CUSTOM5,CUSTOM6,CUSTOM7,CUSTOM8,ITEM
1203,d5,DIGITAL,Internal,,,,,,,,...,,,,,,,,,,


In [170]:
[col for col in df1.columns]

['NAME',
 'TYPE',
 'UNIT',
 'ADDR',
 'RAW_ZERO',
 'RAW_FULL',
 'ENG_ZERO',
 'ENG_FULL',
 'ENG_UNITS',
 'FORMAT',
 'COMMENT',
 'EDITCODE',
 'LINKED',
 'OID',
 'REF1',
 'REF2',
 'DEADBAND',
 'CUSTOM',
 'TAGGENLINK',
 'CLUSTER',
 'HISTORIAN',
 'EQUIP',
 'CUSTOM1',
 'CUSTOM2',
 'CUSTOM3',
 'CUSTOM4',
 'CUSTOM5',
 'CUSTOM6',
 'CUSTOM7',
 'CUSTOM8',
 'ITEM']

In [129]:
project.base.base_tag.to_dbf(df1.to_dict('records'))

In [130]:
project.base.base_tag

In [128]:
df1.to_dict('records')

[{'NAME': 'Cell_2\\Obj3CSWI3\\Pos\\Oper\\ctlVal',
  'TYPE': 'DIGITAL',
  'UNIT': 'PLC_C264_TU',
  'ADDR': 'Cell_2/Obj3CSWI3\\CO\\Pos$Oper$ctlVal',
  'RAW_ZERO': '',
  'RAW_FULL': '',
  'ENG_ZERO': '',
  'ENG_FULL': '',
  'ENG_UNITS': '',
  'FORMAT': '',
  'COMMENT': 'команда увімкнення/вимкнення заземлювача (команда)',
  'EDITCODE': '',
  'LINKED': '',
  'OID': '0x00000010',
  'REF1': '',
  'REF2': '',
  'DEADBAND': '',
  'CUSTOM': '',
  'TAGGENLINK': '',
  'CLUSTER': 'Cluster1',
  'HISTORIAN': 'TRUE',
  'EQUIP': 'Cell_2_TH_6kV_P3U30',
  'CUSTOM1': '',
  'CUSTOM2': '',
  'CUSTOM3': '',
  'CUSTOM4': '',
  'CUSTOM5': '',
  'CUSTOM6': '',
  'CUSTOM7': '',
  'CUSTOM8': '',
  'ITEM': 'Obj3CSWI3\\Pos\\Oper\\ctlVal'},
 {'NAME': 'Cell_2\\Obj2CSWI2\\Pos\\stVal',
  'TYPE': 'STRING',
  'UNIT': 'PLC_C264_TU',
  'ADDR': 'Cell_2/Obj2CSWI2\\ST\\Pos$stVal',
  'RAW_ZERO': '',
  'RAW_FULL': '',
  'ENG_ZERO': '',
  'ENG_FULL': '',
  'ENG_UNITS': '',
  'FORMAT': '',
  'COMMENT': 'положення викотного елеме

In [ ]:
project.create_equip()

In [ ]:
class ItemIEC61850:
    def __init__(self, name_device, node, attribs, type_attr, fs):
        self.node = node
        self.attribs = attribs
        self.attribs_str = '\\'.join(self.attribs)
        self.type_attr = type_attr
        self.fs = fs

        self.name_tag = fr"{self.node}\{self.attribs_str}"
        self.addr = ''



In [134]:
project = Project('tmp1')
plc = 'PLC_C264_TU'

df_dev = pd.read_excel('ListDevice.xlsx', sheet_name='LD')

df_dev['io'] = plc
# df_dev['io'] = df_dev.apply(lambda x: f"{x['name']}_{x['number']}", axis=1)

protocol = 'IEC61850N'
df_io = pd.DataFrame([{'name': plc, 'addr': '', 'protocol': protocol, 'port': 'PORT1_BOARD1'}])
if protocol:
    df_io['addr'] = f"[RUN]:conf61850_{plc}"

project.create_io(df_io)
project.create_equip(df_dev)


In [54]:
project = Project('tmp1')
plc = 'PLC_C264_TU'

df_dev = pd.read_excel('ListDevice.xlsx', sheet_name='P3F30_61850')
df_dev

,node,atrribs,type,fc,comment
0,VO3GGIO99,Ind/t,STRING,ST,автомат кіл 3Uo вимкнено (час)
1,VO3GGIO99,Ind/stVal,STRING,ST,автомат кіл 3Uo вимкнено (режим)
2,VO4GGIO100,Ind/stVal,STRING,ST,автомат оперативних кіл комірки вимкнено (режим)
3,VO4GGIO100,Ind/t,STRING,ST,автомат оперативних кіл комірки вимкнено (час)
4,Obj2CSWI2,Pos/stVal,STRING,ST,положення викотного елементу (режим)
...,...,...,...,...,...
61,U3ppMMXU5,PPV/phsBC/t,STRING,MX,напруга фаз ВC (час)
62,U3ppMMXU5,PPV/phsCA/cVal/mag,STRING,MX,напруга фаз CA (вимірювання)
63,U3ppMMXU5,PPV/phsCA/t,STRING,MX,напруга фаз CA (час)
64,Io1MMXU11,A/neut/cVal/mag,STRING,MX,струм 3Io (вимірювання)


In [53]:
xll = pd.ExcelFile('ListDevice.xlsx')

In [35]:
project = Project('tmp1')
df = project.base.base_tag.dataframe()
df

,NAME,TYPE,UNIT,ADDR,RAW_ZERO,RAW_FULL,ENG_ZERO,ENG_FULL,ENG_UNITS,FORMAT,...,EQUIP,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,CUSTOM5,CUSTOM6,CUSTOM7,CUSTOM8,ITEM
0,i1,INT,Internal,,,,,,,,...,,,,,,,,,,
1,d1,DIGITAL,Internal,,,,,,,,...,,,,,,,,,,
2,r1,REAL,Internal,,,,,,,,...,,,,,,,,,,
3,s1,STRING,Internal,,,,,,,,...,,,,,,,,,,
4,d2,DIGITAL,Internal,,,,,,,,...,,,,,,,,,,
5,d3,DIGITAL,Internal,,,,,,,,...,,,,,,,,,,
6,d4,DIGITAL,Internal,,,,,,,,...,,,,,,,,,,
7,d5,DIGITAL,Internal,,,,,,,,...,,,,,,,,,,
8,q1VO2GGIO98\Ind\t,STRING,,w1VO2GGIO98$ST$Ind$t,,,,,,,...,tt1,,,,,,,,,VO2GGIO98\Ind\t
9,q1VO2GGIO98\Ind\stVal,STRING,,w1VO2GGIO98$ST$Ind$stVal,,,,,,,...,tt1,,,,,,,,,VO2GGIO98\Ind\stVal


In [3]:
project = Project('Proj16850_02122021')
df = project.base.base_tag.dataframe()
df


,NAME,TYPE,UNIT,ADDR,RAW_ZERO,RAW_FULL,ENG_ZERO,ENG_FULL,ENG_UNITS,FORMAT,...,EQUIP,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,CUSTOM5,CUSTOM6,CUSTOM7,CUSTOM8,ITEM
0,RZA3\GGIO1\Ind\t,STRING,C264,Relay/GGIO1$ST$Ind$t,,,,,,,...,RZA3_C264,,,,,,,,,GGIO1\Ind\t
1,RZA3\GGIO1\Ind\stVal,STRING,C264,Relay/GGIO1$ST$Ind$stVal,,,,,,,...,RZA3_C264,,,,,,,,,GGIO1\Ind\stVal
2,RZA3\CSWI1\Pos\stVal,STRING,C264,Relay/CSWI1$ST$Pos$stVal,,,,,,,...,RZA3_C264,,,,,,,,,CSWI1\Pos\stVal
3,RZA3\CSWI1\Pos\t,STRING,C264,Relay/CSWI1$ST$Pos$t,,,,,,,...,RZA3_C264,,,,,,,,,CSWI1\Pos\t
4,RZA3\LLN0\Loc\stVal,STRING,C264,Relay/LLN0$ST$Loc$stVal,,,,,,,...,RZA3_C264,,,,,,,,,LLN0\Loc\stVal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,Cell_5\LO07GGIO83\Ind\t,STRING,C264,Relay/LO07GGIO83$ST$Ind$t,,,,,,,...,Cell_5_P3F30,,,,,,,,,LO07GGIO83\Ind\t
425,Cell_5\LO08GGIO84\Ind\stVal,STRING,C264,Relay/LO08GGIO84$ST$Ind$stVal,,,,,,,...,Cell_5_P3F30,,,,,,,,,LO08GGIO84\Ind\stVal
426,Cell_5\LO08GGIO84\Ind\t,STRING,C264,Relay/LO08GGIO84$ST$Ind$t,,,,,,,...,Cell_5_P3F30,,,,,,,,,LO08GGIO84\Ind\t
427,Cell_5\LO09GGIO85\Ind\stVal,STRING,C264,Relay/LO09GGIO85$ST$Ind$stVal,,,,,,,...,Cell_5_P3F30,,,,,,,,,LO09GGIO85\Ind\stVal


In [60]:
project = Project('Proj16850_02122021')
df = project.base.base_tag.dataframe()

df_eqps = project.base.base_equip.dataframe()
eqps = df_eqps.TYPE.values
# eqps = ['RZA5_C264', 'RZA3_C264', 'Cell_3_P3F30', 'Cell_2_P3U30', 'Cell_11_P3F30', 'Cell_5_P3F30']

for item in eqps:
    eqp = df_eqps[df_eqps['TYPE'] == item]['NAME'].values[0]

    d1 = (df[df['EQUIP'] == eqp]['ITEM'].apply(lambda x: x.split('\\')[0])).values.tolist()
    sr = df[df['EQUIP'] == eqp]['ITEM'].apply(lambda x: x.split('\\')[1:])
    d2 = sr.apply(lambda x: '/'.join(x)).values.tolist()
    d3 = df[df['EQUIP'] == eqp]['TYPE'].values.tolist()
    d4 = df[df['EQUIP'] == eqp]['ADDR'].apply(lambda x: x.split('$')[1] if x else '').values.tolist()
    d5 = df[df['EQUIP'] == eqp]['COMMENT'].values.tolist()

    data = {'node': d1, 'attribs': d2, 'type': d3, 'fc': d4, 'comment': d5}
    df1 = pd.DataFrame(data)

    with ExcelWriter(project.config_xls, mode="a" if os.path.exists(project.config_xls) else "w") as writer:
        df1.to_excel(writer, sheet_name=item + '_61850', index=False)

In [59]:
df_eqps = project.base.base_equip.dataframe()
df_eqps[df_eqps['TYPE'] == 'C264reserve']['NAME'].values[0]

'RZA5_C264'

In [49]:
df2 = project.base.base_equip.dataframe()
df2

,NAME,CLUSTER,TYPE,AREA,LOCATION,COMMENT,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,...,LINKED,EDITCODE,PARAM,COMPOSITE,REFERENCE,DEVSCHED,SCHEDID,ALIAS,CONTENT,HIDDEN
0,RZA5_C264,Cluster1,C264reserve,,,,RZA5\,,,,...,,,,,,,,,,
1,RZA3_C264,Cluster1,C264switch35,,,,RZA3\,,,,...,,,,,,,,,,
2,Cell_3_P3F30,Cluster1,P3F30line6,,,,Cell_3\,,,,...,,,,,,,,,,
3,Cell_2_P3U30,Cluster1,P3U30protectTN,,,,Cell_2\,,,,...,,,,,,,,,,
4,Cell_11_P3F30,Cluster1,P3F30switch6,,,,Cell_11\,,,,...,,,,,,,,,,
5,Cell_5_P3F30,Cluster1,P3F30enter6,,,,Cell_5\,,,,...,,,,,,,,,,


In [56]:
df3 = project.base.base_equip.dataframe()
df3

,NAME,CLUSTER,TYPE,AREA,LOCATION,COMMENT,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,...,LINKED,EDITCODE,PARAM,COMPOSITE,REFERENCE,DEVSCHED,SCHEDID,ALIAS,CONTENT,HIDDEN
0,RZA5_C264,Cluster1,C264reserve,,,,RZA5\,,,,...,,,,,,,,,,
1,RZA3_C264,Cluster1,C264switch35,,,,RZA3\,,,,...,,,,,,,,,,
2,Cell_3_P3F30,Cluster1,P3F30line6,,,,Cell_3\,,,,...,,,,,,,,,,
3,Cell_2_P3U30,Cluster1,P3U30protectTN,,,,Cell_2\,,,,...,,,,,,,,,,
4,Cell_11_P3F30,Cluster1,P3F30switch6,,,,Cell_11\,,,,...,,,,,,,,,,
5,Cell_5_P3F30,Cluster1,P3F30enter6,,,,Cell_5\,,,,...,,,,,,,,,,


,NAME,TYPE
0,RZA3\GGIO1\Ind\t,STRING
1,RZA3\GGIO1\Ind\stVal,STRING
2,RZA3\CSWI1\Pos\stVal,STRING
3,RZA3\CSWI1\Pos\t,STRING
4,RZA3\LLN0\Loc\stVal,STRING
...,...,...
424,Cell_5\LO07GGIO83\Ind\t,STRING
425,Cell_5\LO08GGIO84\Ind\stVal,STRING
426,Cell_5\LO08GGIO84\Ind\t,STRING
427,Cell_5\LO09GGIO85\Ind\stVal,STRING


In [ ]:
project.create_io('Cell', 5)

In [106]:
df1 = pd.DataFrame([{'name': 'C264', 'addr': '[RUN]:conf61850_C264', 'protocol': 'IEC61850N', 'port': 'PORT1_BOARD1'}])
df1

,name,addr,protocol,port
0,C264,[RUN]:conf61850_C264,IEC61850N,PORT1_BOARD1


In [123]:
df = pd.read_excel('ListDevice.xlsx')
df['io'] = 'C264'
# df['io'] = df.apply(lambda x: f"{x['name']}_{x['number']}", axis=1)

,name,number,device,io
0,RZA_T,1,P3T32,RZA_T_1
1,RZA_T,2,P3T32,RZA_T_2
2,Cell_TH_6kV,2,P3U30,Cell_TH_6kV_2
3,Cell_LN_6kV,3,P3F30,Cell_LN_6kV_3
4,Cell_LN_6kV,4,P3F30,Cell_LN_6kV_4
5,Cell_IN_6kV,5,P3F30,Cell_IN_6kV_5
6,Cell_LN_6kV,6,P3F30,Cell_LN_6kV_6
7,Cell_LN_6kV,7,P3F30,Cell_LN_6kV_7
8,Cell_LN_6kV,8,P3F30,Cell_LN_6kV_8
9,Cell_LN_6kV,9,P3F30,Cell_LN_6kV_9


In [108]:
df['io'] = 'C264'
df

,name,number,device,io
0,RZA_T,1,P3T32,C264
1,RZA_T,2,P3T32,C264
2,Cell_TH_6kV,2,P3U30,C264
3,Cell_LN_6kV,3,P3F30,C264
4,Cell_LN_6kV,4,P3F30,C264
5,Cell_IN_6kV,5,P3F30,C264
6,Cell_LN_6kV,6,P3F30,C264
7,Cell_LN_6kV,7,P3F30,C264
8,Cell_LN_6kV,8,P3F30,C264
9,Cell_LN_6kV,9,P3F30,C264


In [4]:
project = Project('ExampleSA')
project.base.base_tag.dataframe()

,NAME,TYPE,UNIT,ADDR,RAW_ZERO,RAW_FULL,ENG_ZERO,ENG_FULL,ENG_UNITS,FORMAT,...,ITEM,HISTORIAN,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,CUSTOM5,CUSTOM6,CUSTOM7,CUSTOM8
0,SA_WindCompass_WindSpeed,REAL,Internal,,0,100,0,100,,,...,WindSpeed,,,,,,,,,
1,SA_WindCompass_WindSpeedAvg,REAL,Internal,,0,100,0,100,,,...,WindSpeedAvg,,,,,,,,,
2,SA_WindCompass_PRHigh,REAL,Internal,,0,100,0,100,,,...,PRHigh,,,,,,,,,
3,SA_WindCompass_PRLow,REAL,Internal,,0,100,0,100,,,...,PRLow,,,,,,,,,
4,SA_WindCompass_WindSpeedThreshold,REAL,Internal,,0,100,0,100,,,...,WindSpeedThreshold,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2448,TopMilk_AC02_LLEnableBypass,DIGITAL,PLC,,,,,,,,...,LLEnableBypass,,,,,,,,,
2449,TopMilk_LS007_HHPMP04BypassCommand,DIGITAL,PLC,,,,,,,,...,HHPMP04BypassCommand,,,,,,,,,
2450,TopMilk_LS007_HHPMP04RemoveBypassCommand,DIGITAL,PLC,,,,,,,,...,HHPMP04RemoveBypassCommand,,,,,,,,,
2451,TopMilk_LS007_HHPMP04BypassStatus,DIGITAL,Cicode,"Bypass_GetValue(""Company.TopMilk.Mixing.LS07.H...",,,,,,,...,HHPMP04BypassStatus,,,,,,,,,


In [10]:
df1 = pd.DataFrame(columns=['node', 'attribs', 'type', 'fc', 'comment'])
df1


,node,attribs,type,fc,comment


,node,attribs,type,fc,comment


,node,attribs,type,fc,comment


In [97]:
df = project.base.base_io.dataframe()

In [51]:
import re

In [94]:
s = 'Cell_2_P3U30_45'
re.findall(r'.*_(\S+)', s)[-1]

'45'

In [40]:
x = BaseProject(os.path.join(r"c:\ProgramData\AVEVA\Citect SCADA 2018 R2\User", 'tmp1', 'equip.DBF'))
x.dataframe()

,NAME,CLUSTER,TYPE,AREA,LOCATION,COMMENT,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,...,LINKED,EDITCODE,PARAM,COMPOSITE,REFERENCE,DEVSCHED,SCHEDID,ALIAS,CONTENT,HIDDEN
0,Cell_2_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_2\,,,,...,,,,,,,,,,
1,Cell_3_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_3\,,,,...,,,,,,,,,,
2,Cell_4_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_4\,,,,...,,,,,,,,,,
3,Cell_5_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_5\,,,,...,,,,,,,,,,
4,Cell_6_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_6\,,,,...,,,,,,,,,,
5,Cell_7_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_7\,,,,...,,,,,,,,,,
6,Cell_8_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_8\,,,,...,,,,,,,,,,
7,Cell_9_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_9\,,,,...,,,,,,,,,,
8,Cell_10_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_10\,,,,...,,,,,,,,,,
9,Cell_11_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_11\,,,,...,,,,,,,,,,


In [137]:
dd = dbf.Table(os.path.join(r"c:\ProgramData\AVEVA\Citect SCADA 2018 R2\User", 'tmp1', 'variable.DBF'),
               codepage='cp1251')
dd.open(mode=dbf.READ_ONLY)
x = dict(zip(dd.field_names, ['' for _ in range(len(dd.field_names))]))
dd.close()
x

{'NAME': '',
 'TYPE': '',
 'UNIT': '',
 'ADDR': '',
 'RAW_ZERO': '',
 'RAW_FULL': '',
 'ENG_ZERO': '',
 'ENG_FULL': '',
 'ENG_UNITS': '',
 'FORMAT': '',
 'COMMENT': '',
 'EDITCODE': '',
 'LINKED': '',
 'OID': '',
 'REF1': '',
 'REF2': '',
 'DEADBAND': '',
 'CUSTOM': '',
 'TAGGENLINK': '',
 'CLUSTER': '',
 'HISTORIAN': '',
 'EQUIP': '',
 'CUSTOM1': '',
 'CUSTOM2': '',
 'CUSTOM3': '',
 'CUSTOM4': '',
 'CUSTOM5': '',
 'CUSTOM6': '',
 'CUSTOM7': '',
 'CUSTOM8': '',
 'ITEM': ''}

In [27]:
st = ''
for t in dd.field_names:
    st += f"self.{t} = ''\n"
print(st)

self.NAME = ''
self.CLUSTER = ''
self.TYPE = ''
self.AREA = ''
self.LOCATION = ''
self.COMMENT = ''
self.CUSTOM1 = ''
self.CUSTOM2 = ''
self.CUSTOM3 = ''
self.CUSTOM4 = ''
self.CUSTOM5 = ''
self.CUSTOM6 = ''
self.CUSTOM7 = ''
self.CUSTOM8 = ''
self.IODEVICE = ''
self.PAGE = ''
self.HELP = ''
self.DEFSTATE = ''
self.SCHEDULED = ''
self.TAGPREFIX = ''
self.TAGGENLINK = ''
self.LINKED = ''
self.EDITCODE = ''
self.PARAM = ''
self.COMPOSITE = ''
self.REFERENCE = ''
self.DEVSCHED = ''
self.SCHEDID = ''
self.ALIAS = ''
self.CONTENT = ''
self.HIDDEN = ''



In [31]:
class BaseProject:

    def __init__(self, file_name):
        self.filename = filename


In [32]:
x = Clss()

In [34]:
x.filename = 'jfjdklj'
x.__dict__

{'NAME': '',
 'CLUSTER': '',
 'TYPE': '',
 'AREA': '',
 'LOCATION': '',
 'COMMENT': '',
 'CUSTOM1': '',
 'CUSTOM2': '',
 'CUSTOM3': '',
 'CUSTOM4': '',
 'CUSTOM5': '',
 'CUSTOM6': '',
 'CUSTOM7': '',
 'CUSTOM8': '',
 'IODEVICE': '',
 'PAGE': '',
 'HELP': '',
 'DEFSTATE': '',
 'SCHEDULED': '',
 'TAGPREFIX': '',
 'TAGGENLINK': '',
 'LINKED': '',
 'EDITCODE': '',
 'PARAM': '',
 'COMPOSITE': '',
 'REFERENCE': '',
 'DEVSCHED': '',
 'SCHEDID': '',
 'ALIAS': '',
 'CONTENT': '',
 'HIDDEN': '',
 'filename': 'jfjdklj'}

In [77]:
dd.open(mode=dbf.READ_ONLY)

dbf.Table('c:\\ProgramData\\AVEVA\\Citect SCADA 2018 R2\\User\\tmp1\\units.DBF', status=<DbfStatus.READ_ONLY: 1>)

In [12]:
dd.close()

In [68]:
project.show_io()

,SERVER,NAME,NUMBER,ADDR,PROTOCOL,PORT,MODE,LOGWRITE,LOGREAD,CACHE,...,TAGGENTEMP,PRIORITY,BGPOLL,BGPOLLRATE,PERSFREQ,PERSPATH,MINUPDATE,STALEPERIO,READONLY,EXCLUSIVE
0,IOServer1,Internal,1,,OFSOPC,,Primary,,,,...,,1,,,,,,,,
1,IOServer1,Cicode,2,,CICODE,,Primary,,,,...,,1,,,,,,,,


In [57]:
project = Project('tmp1')
project.write_io()

ValueError: invalid literal for int() with base 10: '        '

Traceback (most recent call last):
  File "_pydevd_bundle\pydevd_cython_win32_39_64.pyx", line 1034, in _pydevd_bundle.pydevd_cython_win32_39_64.PyDBFrame.trace_dispatch
  File "C:\Program Files\JetBrains\PyCharm 213.4631.9\plugins\python\helpers-pro\jupyter_debug\pydev_jupyter_plugin.py", line 144, in cmd_step_over
    if _is_inside_jupyter_cell(frame, pydb):
  File "C:\Program Files\JetBrains\PyCharm 213.4631.9\plugins\python\helpers-pro\jupyter_debug\pydev_jupyter_plugin.py", line 209, in _is_inside_jupyter_cell
    if is_cell_filename(filename):
  File "C:\Program Files\JetBrains\PyCharm 213.4631.9\plugins\python\helpers-pro\jupyter_debug\pydev_jupyter_plugin.py", line 220, in is_cell_filename
    ipython_shell = get_ipython()
NameError: name 'get_ipython' is not defined


In [20]:
# Удаление строки
rec = table[-1]
dbf.delete(rec)
table.pack()

In [11]:
name_file = r"units.DBF"
filename = os.path.join(path_projects, name_projects, name_file)

db_in_dbf = DBF(filename, encoding='cp1251')
df = pd.DataFrame(db_in_dbf)
df


,SERVER,NAME,NUMBER,ADDR,PROTOCOL,PORT,MODE,LOGWRITE,LOGREAD,CACHE,...,TAGGENTEMP,PRIORITY,BGPOLL,BGPOLLRATE,PERSFREQ,PERSPATH,MINUPDATE,STALEPERIO,READONLY,EXCLUSIVE
0,IOServer1,Internal,1,,OFSOPC,,Primary,,,,...,,1,,,,,,,,
1,IOServer1,Cicode,2,,CICODE,,Primary,,,,...,,1,,,,,,,,


In [10]:
name_file = r"variable.DBF"
filename = os.path.join(path_projects, name_projects, name_file)

db_in_dbf = DBF(filename, encoding='cp1251')
df = pd.DataFrame(db_in_dbf)
df

,NAME,TYPE,UNIT,ADDR,RAW_ZERO,RAW_FULL,ENG_ZERO,ENG_FULL,ENG_UNITS,FORMAT,...,EQUIP,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,CUSTOM5,CUSTOM6,CUSTOM7,CUSTOM8,ITEM
0,Cell_2\VO3GGIO99\Ind\t,STRING,Cell_2,Relay/VO3GGIO99$ST$Ind$t,,,,,,,...,Cell_2_P3U30,,,,,,,,,VO3GGIO99\Ind\t
1,Cell_2\VO3GGIO99\Ind\stVal,STRING,Cell_2,Relay/VO3GGIO99$ST$Ind$stVal,,,,,,,...,Cell_2_P3U30,,,,,,,,,VO3GGIO99\Ind\stVal
2,Cell_2\VO4GGIO100\Ind\stVal,STRING,Cell_2,Relay/VO4GGIO100$ST$Ind$stVal,,,,,,,...,Cell_2_P3U30,,,,,,,,,VO4GGIO100\Ind\stVal
3,Cell_2\VO4GGIO100\Ind\t,STRING,Cell_2,Relay/VO4GGIO100$ST$Ind$t,,,,,,,...,Cell_2_P3U30,,,,,,,,,VO4GGIO100\Ind\t
4,Cell_2\Obj2CSWI2\Pos\stVal,STRING,Cell_2,Relay/Obj2CSWI2$ST$Pos$stVal,,,,,,,...,Cell_2_P3U30,,,,,,,,,Obj2CSWI2\Pos\stVal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,Cell_20\U3ppMMXU5\PPV\phsBC\t,STRING,Cell_20,Relay/U3ppMMXU5$MX$PPV$phsBC$t,,,,,,,...,Cell_20_P3U30,,,,,,,,,U3ppMMXU5\PPV\phsBC\t
699,Cell_20\U3ppMMXU5\PPV\phsCA\cVal,STRING,Cell_20,Relay/U3ppMMXU5$MX$PPV$phsCA$cVal,,,,,,,...,Cell_20_P3U30,,,,,,,,,U3ppMMXU5\PPV\phsCA\cVal
700,Cell_20\U3ppMMXU5\PPV\phsCA\t,STRING,Cell_20,Relay/U3ppMMXU5$MX$PPV$phsCA$t,,,,,,,...,Cell_20_P3U30,,,,,,,,,U3ppMMXU5\PPV\phsCA\t
701,Cell_20\UovMMXU1\PhV\neut\cVal,STRING,Cell_20,Relay/UovMMXU1$MX$PhV$neut$cVal,,,,,,,...,Cell_20_P3U30,,,,,,,,,UovMMXU1\PhV\neut\cVal


In [7]:
name_file = r"equip.DBF"
filename = os.path.join(path_projects, name_projects, name_file)

db_in_dbf = DBF(filename, encoding='cp1251')
df = pd.DataFrame(db_in_dbf)
df

,NAME,CLUSTER,TYPE,AREA,LOCATION,COMMENT,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,...,LINKED,EDITCODE,PARAM,COMPOSITE,REFERENCE,DEVSCHED,SCHEDID,ALIAS,CONTENT,HIDDEN
0,Cell_2_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_2\,,,,...,,,,,,,,,,
1,Cell_3_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_3\,,,,...,,,,,,,,,,
2,Cell_4_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_4\,,,,...,,,,,,,,,,
3,Cell_5_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_5\,,,,...,,,,,,,,,,
4,Cell_6_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_6\,,,,...,,,,,,,,,,
5,Cell_7_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_7\,,,,...,,,,,,,,,,
6,Cell_8_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_8\,,,,...,,,,,,,,,,
7,Cell_9_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_9\,,,,...,,,,,,,,,,
8,Cell_10_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_10\,,,,...,,,,,,,,,,
9,Cell_11_P3U30,Cluster1,P3U30,,,Защита ячейки 2,Cell_11\,,,,...,,,,,,,,,,


In [ ]:
path_projects = r"c:\ProgramData\AVEVA\Citect SCADA 2018 R2\User"
name_projects = "tmp1"


# name_file = r"equip.DBF"
# name_file = r"pgdynobj.DBF"
# filename = os.path.join(path_projects, name_projects, name_file)


class TypeDevice:
    P3U30 = 'P3U30'
    P3F30 = 'P3F30'
    P3T32 = 'P3T32'


class ProjectBases:

    def __init__(self, path_project):
        self.path_project = path_project
        self.base_equip = BaseProject(os.path.join(self.path_project, 'equip.DBF'))
        self.base_io = BaseProject(os.path.join(self.path_project, 'units.DBF'))
        self.base_tag = BaseProject(os.path.join(self.path_project, 'variable.DBF'))


class BaseProject:

    def __init__(self, file_name):
        self.filename = file_name

    def get_dbf(self):
        db = None
        with dbf.Table(self.filename, codepage='cp1251') as tb:
            db = tb.open(mode=dbf.READ_ONLY)
        return db

    def to_dbf(self, params_list):
        with dbf.Table(self.filename, codepage='cp1251') as tb:
            tb.open(mode=dbf.READ_WRITE)
            for params in params_list:
                tb.append(params)

    def clear_dbf(self):
        with dbf.Table(self.filename, codepage='cp1251') as tb:
            tb.open(mode=dbf.READ_WRITE)
            tb.zap()

    def dataframe(self):
        return pd.DataFrame(DBF(self.filename, encoding='cp1251'))


class P3U30:
    def __init__(self, name_project, name_cluster='Cluster1', name_server='IOServer1'):
        self.name = 'P3U30'
        self.name_projects = name_project
        self.path_project = os.path.join(self.path_projects, self.name_projects)

        self.name_files = {'Оборудование': 'equip.DBF', 'Теги': 'variable.DBF', 'IO': 'units.DBF'}
        self.type_devices = TypeDevice()
        self.type_device_current = ''
        self.cluster = name_cluster
        self.server = name_server
        self.base = ProjectBases(self.path_project)


class Project:
    def __init__(self, name_project, name_cluster='Cluster1', name_server='IOServer1'):
        self.path_projects = r"c:\ProgramData\AVEVA\Citect SCADA 2018 R2\User"
        self.name_projects = name_project
        self.path_project = os.path.join(self.path_projects, self.name_projects)

        self.config_xls = self.get_path_file('ListDevice.xlsx')

        self.name_files = {'Оборудование': 'equip.DBF', 'Теги': 'variable.DBF', 'IO': 'units.DBF'}
        self.type_devices = TypeDevice()
        self.type_device_current = ''
        self.cluster = name_cluster
        self.server = name_server
        self.base = ProjectBases(self.path_project)

        self.df_io = None
        try:
            self.df_io = pd.read_excel(self.config_xls, sheet_name='IO')
        except:
            pass

        self.df_equips = None
        try:
            self.df_equips = pd.read_excel(self.config_xls, sheet_name='LD')
        except:
            pass

    def create_io(self):

        if self.df_io is not None:

            self.df_io['port'] = 'PORT1_BOARD1'
            addr = ''

            params_list = []

            df_tmp = self.base.base_io.dataframe()
            try:
                numb = int(df_tmp.iloc[-1]['NUMBER'])
            except:
                numb = 1

            df_io_exist = self.base.base_io.dataframe()

            for i in range(len(self.df_io)):
                io = self.df_io.iloc[i]

                if io['name'] not in df_io_exist['NAME'].values:
                    numb += 1

                    if io['protocol'] == 'IEC61850N':
                        addr = f"[RUN]:conf61850_{io['name']}"
                        self.create_config61850(f"conf61850_{io['name']}.xml")

                    values = {'SERVER': self.server,
                              'NAME': io['name'],
                              'NUMBER': f'{numb}',
                              'ADDR': addr,
                              'PROTOCOL': io['protocol'],
                              'PORT': io['port'],
                              'MODE': 'Primary',
                              'LOGWRITE': '',
                              'LOGREAD': '',
                              'CACHE': '',
                              'CACHETIME': '',
                              'COMMENT': '',
                              'PROTOCOLID': '',
                              'LINKED': '',
                              'EXTERNDB': '',
                              'DRIVERID': '',
                              'CONNSTRING': '',
                              'REFRESH': '',
                              'TAGPREFIX': '',
                              'LASTUPDATE': '',
                              'REMOTE': '',
                              'REMOTEWRIT': '',
                              'TIME': '',
                              'PERIOD': '',
                              'PHONE': '',
                              'CALLERID': '',
                              'ONCONNECT': '',
                              'ONHANGUP': '',
                              'ONBROWSE': '',
                              'LIVEUPDATE': '',
                              'LASTVARMOD': '',
                              'MEMORY': 'FALSE',
                              'PERSIST': '',
                              'TAGGEN': '',
                              'TAGGENTEMP': '',
                              'PRIORITY': '1',
                              'BGPOLL': '',
                              'BGPOLLRATE': '',
                              'PERSFREQ': '',
                              'PERSPATH': '',
                              'MINUPDATE': '',
                              'STALEPERIO': '',
                              'READONLY': '',
                              'EXCLUSIVE': ''}
                    params_list.append(values)

            self.base.base_io.to_dbf(params_list)

    def create_equip(self):
        if self.df_equips is not None:

            self.df_equips['io'] = self.df_io.iloc[0]['name']  #TODO

            params_list = []

            df_equip_exist = self.base.base_equip.dataframe()

            for i in range(len(self.df_equips)):
                equip = self.df_equips.iloc[i]

                values = {'NAME': f"{equip['name']}_{equip['number']}_{equip['spec']}_{equip['device']}",
                          'CLUSTER': self.cluster,
                          'TYPE': equip['device'],
                          'AREA': '',
                          'LOCATION': '',
                          'COMMENT': equip['comment'],
                          'CUSTOM1': f"{equip['io']}\\",
                          'CUSTOM2': '',
                          'CUSTOM3': '',
                          'CUSTOM4': '',
                          'CUSTOM5': '',
                          'CUSTOM6': '',
                          'CUSTOM7': '',
                          'CUSTOM8': '',
                          'IODEVICE': equip['io'],
                          'PAGE': '',
                          'HELP': '',
                          'DEFSTATE': '',
                          'SCHEDULED': '',
                          'TAGPREFIX': f"{equip['name']}_{equip['number']}/",
                          'TAGGENLINK': '',
                          'LINKED': '',
                          'EDITCODE': '',
                          'PARAM': '',
                          'COMPOSITE': '',
                          'REFERENCE': '',
                          'DEVSCHED': '',
                          'SCHEDID': '',
                          'ALIAS': '',
                          'CONTENT': '',
                          'HIDDEN': ''}

                if len(df_equip_exist) == 0:
                    params_list.append(values)
                else:
                    if values['NAME'] not in df_equip_exist['NAME'].values:
                        params_list.append(values)

                sheet_name = equip['device'] + '_61850'
                df_dev = pd.read_excel(self.config_xls, sheet_name=sheet_name)
                df_dev['io'] = equip['io']
                df_dev['LD'] = f"{equip['name']}_{equip['number']}"
                df_dev['equip'] = values['NAME']
                self.create_tag(df_dev, type='IEC61850')

            self.base.base_equip.to_dbf(params_list)

    def create_tag_61850(self, df, replace=None):
        df_tags_exist = self.base.base_tag.dataframe()

        names = []
        if len(df_tags_exist) != 0:
            names = list(df_tags_exist['NAME'].values)

        df_tags = pd.DataFrame()
        if replace and len(df_tags_exist) != 0:
            df_tags = df_tags_exist

        if replace:
            self.base.base_tag.clear_dbf()

        for i in range(len(df)):
            dev = df.iloc[i]

            attribs = dev['attribs'].split('/')
            name_tag_attribs = '\\'.join(attribs)
            values = {'NAME': f"{dev['LD']}\\{dev['node']}\\{name_tag_attribs}",
                      'TYPE': dev['type'],
                      'UNIT': dev['io'],
                      'ADDR': f"{dev['LD']}/{dev['node']}${dev['fc']}\\{'$'.join(attribs)}",
                      'RAW_ZERO': '',
                      'RAW_FULL': '',
                      'ENG_ZERO': '',
                      'ENG_FULL': '',
                      'ENG_UNITS': '',
                      'FORMAT': '',
                      'COMMENT': dev['comment'],
                      'EDITCODE': '',
                      'LINKED': '',
                      'OID': '',
                      'REF1': '',
                      'REF2': '',
                      'DEADBAND': '',
                      'CUSTOM': '',
                      'TAGGENLINK': '',
                      'CLUSTER': self.cluster,
                      'HISTORIAN': 'TRUE',
                      'EQUIP': dev['equip'],
                      'CUSTOM1': '',
                      'CUSTOM2': '',
                      'CUSTOM3': '',
                      'CUSTOM4': '',
                      'CUSTOM5': '',
                      'CUSTOM6': '',
                      'CUSTOM7': '',
                      'CUSTOM8': '',
                      'ITEM': f"{dev['node']}\\{name_tag_attribs}"}

            if values['NAME'] not in names:
                df_tags = pd.concat([df_tags, pd.DataFrame([values])], ignore_index=True)
                names.append(values['NAME'])
            else:
                if replace:
                    df_tags.loc[df_tags['NAME'] == values['NAME'], :] = list(values.values())

        # if len(df_tags_exist)==0:
        #     # params_list.append(values)
        #     df_tags = pd.DataFrame([values])
        # else:
        #     if values['NAME'] not in df_tags_exist['NAME'].values:
        #         # params_list.append(values)
        #         df_tags = pd.concat([df_tags, pd.DataFrame([values])], ignore_index=True)
        #     else:
        #         if replace:
        #             df_tags.loc[df_tags['NAME']==values['NAME'], :] = list(values.values())
        #             # dev_repl = df_tags_exist[df_tags_exist['NAME'] == dev['name']]
        #             # for col in df_tags_exist.columns:
        #             #     if col in dev.index.values:
        #             #         dev_repl[col] = dev[col]
        #             #     else:
        #             #         dev_repl[col] = ''

        self.base.base_tag.to_dbf(df_tags.to_dict('records'))

    def create_tag_simple(self, df, replace=None):
        params_list = []

        df_tags_exist = self.base.base_tag.dataframe()

        names = []
        if len(df_tags_exist) != 0:
            names = list(df_tags_exist['NAME'].values)

        df_tags = pd.DataFrame()
        if replace and len(df_tags_exist) != 0:
            df_tags = df_tags_exist

        if replace:
            # df_old = self.base.base_tag.dataframe()
            self.base.base_tag.clear_dbf()
            # params_list = df_old.to_dict('records')

        for i in range(len(df)):
            dev = df.iloc[i]
            values = {'NAME': dev['name'],
                      'TYPE': dev['type'],
                      'UNIT': dev['io'],
                      'ADDR': "",
                      'RAW_ZERO': '',
                      'RAW_FULL': '',
                      'ENG_ZERO': '',
                      'ENG_FULL': '',
                      'ENG_UNITS': '',
                      'FORMAT': '',
                      'COMMENT': dev['comment'],
                      'EDITCODE': '',
                      'LINKED': '',
                      'OID': '',
                      'REF1': '',
                      'REF2': '',
                      'DEADBAND': '',
                      'CUSTOM': '',
                      'TAGGENLINK': '',
                      'CLUSTER': self.cluster,
                      'HISTORIAN': 'TRUE',
                      'EQUIP': '',
                      'CUSTOM1': '',
                      'CUSTOM2': '',
                      'CUSTOM3': '',
                      'CUSTOM4': '',
                      'CUSTOM5': '',
                      'CUSTOM6': '',
                      'CUSTOM7': '',
                      'CUSTOM8': '',
                      'ITEM': ''}

            if values['NAME'] not in names:
                df_tags = pd.concat([df_tags, pd.DataFrame([values])], ignore_index=True)
                names.append(values['NAME'])
            else:
                if replace:
                    df_tags.loc[df_tags['NAME'] == values['NAME'], :] = list(values.values())

            # if len(df_tags_exist) == 0:
            #     # params_list.append(values)
            #     df_tags = pd.DataFrame([values])
            # else:
            #     if values['NAME'] not in df_tags_exist['NAME'].values:
            #         # params_list.append(values)
            #         df_tags = pd.concat([df_tags_exist, pd.DataFrame([values])], ignore_index=True)
            #     else:
            #         if replace:
            #             df_tags.loc[df_tags['NAME'] == values['NAME'], :] = list(values.values())
            #             # dev_repl = df_tags_exist[df_tags_exist['NAME'] == dev['name']]
            #             # for col in df_tags_exist.columns:
            #             #     if values[col] != '':
            #             #         # df_tags_exist[df_tags_exist['NAME'] == dev['name']].loc[:,col] = dev[col]
            #             #         df_tags_exist[col].where(~(df_tags_exist['NAME'] == dev['name']), other=values[col], inplace=True)
            #             #
            #             #     else:
            #             #         # df_tags_exist[df_tags_exist['NAME'] == dev['name']].loc[:,col] = ''
            #             #         df_tags_exist[col].where(~(df_tags_exist['NAME'] == dev['name']), other='', inplace=True)

        # self.base.base_tag.to_dbf(params_list)
        self.base.base_tag.to_dbf(df_tags.to_dict('records'))

    def create_tag_values(self, df, type='simple'):
        '''

        :param df:
        :param type: 'simple', 'IEC61850'
        :return:
        '''
        dev = df

        values = {}

        if type == 'simple':
            values = {'NAME': dev['name'],
                      'TYPE': dev['type'],
                      'UNIT': dev['io'],
                      'ADDR': "",
                      'RAW_ZERO': '',
                      'RAW_FULL': '',
                      'ENG_ZERO': '',
                      'ENG_FULL': '',
                      'ENG_UNITS': '',
                      'FORMAT': '',
                      'COMMENT': dev['comment'],
                      'EDITCODE': '',
                      'LINKED': '',
                      'OID': '',
                      'REF1': '',
                      'REF2': '',
                      'DEADBAND': '',
                      'CUSTOM': '',
                      'TAGGENLINK': '',
                      'CLUSTER': self.cluster,
                      'HISTORIAN': 'TRUE',
                      'EQUIP': '',
                      'CUSTOM1': '',
                      'CUSTOM2': '',
                      'CUSTOM3': '',
                      'CUSTOM4': '',
                      'CUSTOM5': '',
                      'CUSTOM6': '',
                      'CUSTOM7': '',
                      'CUSTOM8': '',
                      'ITEM': ''}

        if type == 'IEC61850':
            attribs = dev['attribs'].split('/')
            name_tag_attribs = '\\'.join(attribs)
            values = {'NAME': f"{dev['LD']}\\{dev['node']}\\{name_tag_attribs}",
                      'TYPE': dev['type'],
                      'UNIT': dev['io'],
                      'ADDR': f"{dev['LD']}/{dev['node']}${dev['fc']}\\{'$'.join(attribs)}",
                      'RAW_ZERO': '',
                      'RAW_FULL': '',
                      'ENG_ZERO': '',
                      'ENG_FULL': '',
                      'ENG_UNITS': '',
                      'FORMAT': '',
                      'COMMENT': dev['comment'],
                      'EDITCODE': '',
                      'LINKED': '',
                      'OID': '',
                      'REF1': '',
                      'REF2': '',
                      'DEADBAND': '',
                      'CUSTOM': '',
                      'TAGGENLINK': '',
                      'CLUSTER': self.cluster,
                      'HISTORIAN': 'TRUE',
                      'EQUIP': dev['equip'],
                      'CUSTOM1': '',
                      'CUSTOM2': '',
                      'CUSTOM3': '',
                      'CUSTOM4': '',
                      'CUSTOM5': '',
                      'CUSTOM6': '',
                      'CUSTOM7': '',
                      'CUSTOM8': '',
                      'ITEM': f"{dev['node']}\\{name_tag_attribs}"}

        return values

    def create_tag(self, df, type='simple', replace=None):
        '''

        :param df:
        :param type: 'simple', 'IEC61850'
        :param replace:
        :return:
        '''
        df_tags_exist = self.base.base_tag.dataframe()

        names = []
        if len(df_tags_exist) != 0:
            names = list(df_tags_exist['NAME'].values)

        df_tags = pd.DataFrame()
        if replace and len(df_tags_exist) != 0:
            df_tags = df_tags_exist

        if replace:
            self.base.base_tag.clear_dbf()

        for i in range(len(df)):
            dev = df.iloc[i]
            values = self.create_tag_values(dev, type=type)

            if values['NAME'] not in names:
                df_tags = pd.concat([df_tags, pd.DataFrame([values])], ignore_index=True)
                names.append(values['NAME'])
            else:
                if replace:
                    df_tags.loc[df_tags['NAME'] == values['NAME'], :] = list(values.values())

        self.base.base_tag.to_dbf(df_tags.to_dict('records'))

    def create_config61850(self, name_file):
        xml_doc = f"""
            <?xml version="1.0" encoding="utf-8"?>
            <ScadaDevice xmlns="http://www.schneider-electric.com/SCADA/Drivers/IEC61850/DeviceConfig/v1/">
               <IPConnection>
                  <IP>127.0.0.1</IP>
               </IPConnection>
               <IED>TEMPLATE</IED>
               <LogicalDevice Name="Relay">
                  <BRCB>brcbEV1</BRCB>
                  <BRCB>brcbMX01</BRCB>
                  <URCB>urcbMX01</URCB>
               </LogicalDevice>
            </ScadaDevice>
            """

        soup = BeautifulSoup(xml_doc, 'lxml-xml')
        xml = soup.prettify()
        with open(self.get_path_file(name_file), "w") as file:
            file.write(xml)

    def get_path_file(self, name_file):
        return os.path.join(self.path_project, name_file)

